# Notebook : Création du fichier de travail
On va créer notre fichier de travail en réalisant des jointures, ainsi que des calculs d'agrégats sur ces jointures, afin d'extraire les informations qui nous intéressent. 
## Sommaire <a name=0 ></a>
- [A - Etapes préliminaires](#A)
- [B - Création des variables - Feature Engineering](#B)
    - [B.1 - Variables "Générales"](#B.1)
        - [B.1.a - Géolocalisation des clients (geolocation_lng, geolocation_lat)](#B.1.a)
        - [B.1.b - Nombre total de commandes effectuées (nb_orders)](#B.1.b)
        - [B.1.c - Note de satisfaction moyenne (mean_review_score)](#B.1.c)
        - [B.1.d - Moyen de paiement préféré (hors vouchers/bons d'achats)](#B.1.d)
        - [B.1.e - Nombre de bon d'achats utilisés (nb_voucher)](#B.1.e)
        - [B.1.f - Montant total des bon d'achats utilisés (voucher_value)](#B.1.f)
        - [B.1.g - Nombre de versements moyen (mean_payment_installments)](#B.1.g)

    - [B.2 - Variables "RFM" ](#B.2)
        - [B.2.a - Récence : Nombre de jours depuis la dernière commande (nb_days_last_order)](#B.2.a)
        - [B.2.b - Fréquence](#B.2.b)
            - [B.2.b1 - Nombre de jours moyen entre 2 commandes (delta_moy)](#B.2.b1)
            - [B.2.b2 - Catégories des produits les plus commandés (most_freq_cat_name)](#B.2.b2)
        - [B.2.c - Montant :](#B.2.c)
            - [B.2.c1 - Montant total des commandes (payment_value)](#B.2.c1)
            - [B.2.c2 - Panier :](#B.2.c2)
                - [B.2.c2.a - Montant du panier moyen (average_basket)](#B.2.c2.a)
                - [B.2.c2.b - Montant du panier minimum (min_basket)](#B.2.c2.b)
                - [B.2.c2.c - Montant du panier maximum (max_basket)](#B.2.c2.c)
                 
            - [B.2.c3 - Frais de port : ](#B.2.c3)
                - [B.2.c3.a - Total des frais de port (sum_fdp)](#B.2.c3.a)
                - [B.2.c3.b - Frais de port moyens (mean_fdp)](#B.2.c3.b)
                - [B.2.c3.c - Frais de port minimum (fdp_min)](#B.2.c3.c)
                - [B.2.c3.d - Frais de port maximum (fdp_max)](#B.2.c3.d)
- [C - Enregistrement du fichier de travail](#A)

## Chargement des bibliothèques 

In [1]:
%matplotlib inline  
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import seaborn as sns
import datetime
from IPython.display import Image

## Définition de fonctions

In [2]:
def recap(df):
    """
    - Paramètre : Dataframe 
    - Retourne : Dataframe concatenant 'describe' et 'dtypes' pour chaques colonnes du dataframe en entrée 
    """
    return pd.concat([df.describe(include='all'), df.dtypes.to_frame(name='type').T])

def moda(df,col):
    """
    - Paramètres : 
        - Dataframe
        - Colonne du Dataframe
    - Retourne : Dataframe des modalités de la colonne et du dataframe en entrée avec leur fréquences et leurs effectifs respectifs
    """
    #on comptes le nombre de valeurs dans la colonne
    eff = df[col].value_counts() 
    #on récupères les modalités que prends la variable
    moda = eff.index 
    
    #on crée un DataFrame avec toutes les modalités de la variable
    tab_moda = pd.DataFrame(moda, columns = [col])
    #On ajoute une colonne au DataFrame contenant le nombre d'occurence de la modalité
    tab_moda["n"] = eff.values
    #on ajoute une colonne au DataFrame colculant la fréquence d'apparition de la modalité
    tab_moda["f"] = tab_moda["n"]/sum(tab_moda["n"])*100 
    return tab_moda

## Chargement des fichiers 

In [3]:
customers = pd.read_csv("Data/Clean/customers.csv", dtype={'customer_zip_code_prefix':str})
geoloc = pd.read_csv("Data/Clean/geoloc.csv", dtype={'geolocation_zip_code_prefix':str})
order_items = pd.read_csv("Data/Clean/order_items.csv", dtype={'order_item_id':str})
order_payments = pd.read_csv("Data/Clean/order_payments.csv")
order_reviews = pd.read_csv("Data/Clean/order_reviews.csv")
orders = pd.read_csv("Data/Clean/orders.csv")
products = pd.read_csv("Data/Clean/products.csv")
sellers = pd.read_csv("Data/Clean/sellers.csv", dtype={'seller_zip_code_prefix':str})
cat_name = pd.read_csv("Data/Clean/cat_name.csv")

In [4]:
orders['order_purchase_timestamp']=pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at']=pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date']=pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date']=pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date']=pd.to_datetime(orders['order_estimated_delivery_date'])

## A - Etapes préliminaires <a name=A ></a>
On va maintenant créer le fichier de travail avec les variables qui nous intéressent. 
### customer_unique_id
Sachant que l'on veut faire un segmentation client, l'individus doit être un client unique. On va donc récupérer les ID de tout les clients uniques à notre disposition :

In [5]:
work=customers[['customer_unique_id']]
work=work.drop_duplicates(keep='first')
work

,customer_unique_id
0,861eff4711a542e4b93843c6dd7febb0
1,290c77bc529b7ac935b93aa66c333dc3
2,060e732b5b29e8181a18229c7b0b2b5e
3,259dac757896d24d7702b9acbbff3f3c
4,345ecd01c38d18a9036ed96c73b8d066
...,...
99436,1a29b476fee25c95fbafc67c5ac95cf8
99437,d52a67c98be1cf6a5c84435bd38d095d
99438,e9f50caf99f032f0bf3c55141f019d99
99439,73c2643a0a458b49f58cea58833b192e


On récupère donc 96096 clients uniques.
### Période d'observation
Avant de passer à la création de features on va définir une période d'observation. Regardons la période couverte par les données à notre disposition :

In [6]:
orders['order_purchase_timestamp']=pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_purchase_timestamp'].max()

Timestamp('2018-10-17 17:30:18')

In [7]:
orders['order_purchase_timestamp'].min()

Timestamp('2016-09-04 21:15:19')

La période d'observation contient 3 années, pas toutes complètes : 
- 2016 : 09 -> 12
- 2017 : 01 -> 12
- 2018 : 01 -> 10
L'année 2017 est complète on va donc la prendre comme année de référence pour l'analyse. On va donc considérer que l'on effectue une segmentation client pour l'année 2017.

#### Définition date de début & date de fin de la période considérée

In [8]:
#-----1er_fichier :
#date_start = '2017-01-01'
#date_end = '2017-12-31'
#-----2ème_fichier :
date_start = '2017-02-01'
date_end = '2018-01-31'
#-----3ème_fichier :
#date_start = '2017-04-01'
#date_end = '2018-03-31'

In [9]:
nom_fichier = date_start + '_to_' + date_end +'.csv'
nom_fichier

'2017-02-01_to_2018-01-31.csv'

C'est donc cette période qui sera considérée pour le calcul des variables. <br> On va aussi en profiter pour nommer le fichier de travail. En effet si on a besoin de créer de nouveaux fichiers portant sur des périodes de temps différentes il faut que l'on pense à ne pas écraser les autres fichiers et donc penser à renommer avant de l'enregistrer.

In [10]:
date_start = datetime.datetime.strptime(date_start, '%Y-%m-%d')
date_end = datetime.datetime.strptime(date_end, '%Y-%m-%d')

## B - Création des variables - Feature Engineering <a name=B ></a>
Maintenant que l'on a effectuer les étapes préparatoires on peut passer à la création des variables. J'ai décidé de scinder cette étapes en 2 sous-étapes :
- Variables "Générales" 
- Variables liées à une segmentation RFM (Récence-Fréquence-Montant) : qui est une méthode de segmentation client

## B.1 - Variables "Générales" : <a name=B.1 ></a>
### B.1.a - Géolocalisation des clients (geolocation_lng, geolocation_lat) : <a name=B.1.a ></a>
On veut récupérer une longitude et une latitude pour chaque client, on va donc utiliser les fichiers 'geoloc' et 'customers' :

In [11]:
geoloc.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.545621,-46.639292,sao paulo,SP
1,01046,-23.546081,-46.644820,sao paulo,SP
2,01046,-23.546129,-46.642951,sao paulo,SP
3,01041,-23.544392,-46.639499,sao paulo,SP
4,01035,-23.541578,-46.641607,sao paulo,SP


On observe que pour une même ville on peut avoir plusieurs zip_code et plusieurs longitudes/lattitudes. On va donc attribuer une unique paire de longitude-lattitude à une ville, qui seras la moyenne des longitudes & lattitudes de la ville :

In [12]:
#on récupère toutes les villes 
geoloc['geolocation_city']=geoloc['geolocation_city'].str.upper()
geoloc_city = geoloc[['geolocation_city']]
geoloc_city = geoloc_city.drop_duplicates(keep='first')
geoloc_city = geoloc_city.reset_index(drop=True)
geoloc_city

,geolocation_city
0,SAO PAULO
1,SÃO PAULO
2,SAO BERNARDO DO CAMPO
3,JUNDIAÍ
4,TABOÃO DA SERRA
...,...
8006,IBIAÇÁ
8007,SANTA CECILIA DO SUL
8008,CIRÍACO
8009,ESTAÇÃO


In [13]:
#moyenne des longitudes 
geo_long=pd.DataFrame(geoloc[['geolocation_lat','geolocation_lng','geolocation_city']].groupby('geolocation_city')['geolocation_lng'].mean())
geo_long=geo_long.reset_index()
geo_long

,geolocation_city,geolocation_lng
0,* CIDADE,-49.334374
1,...ARRAIAL DO CABO,-42.029834
2,4O. CENTENARIO,-53.069433
3,4º CENTENARIO,-53.074723
4,ABADIA DE GOIAS,-49.438089
...,...,...
8006,ÁUREA,-52.055076
8007,ÂNGULO,-51.917294
8008,ÉRICO CARDOSO,-42.134282
8009,ÓBIDOS,-55.516370


In [14]:
#moyenne des lattitudes 
geo_lat=pd.DataFrame(geoloc[['geolocation_lat','geolocation_lng','geolocation_city']].groupby('geolocation_city')['geolocation_lat'].mean())
geo_lat=geo_lat.reset_index()
geo_lat

,geolocation_city,geolocation_lat
0,* CIDADE,-25.571748
1,...ARRAIAL DO CABO,-22.969370
2,4O. CENTENARIO,-24.271860
3,4º CENTENARIO,-24.279801
4,ABADIA DE GOIAS,-16.767063
...,...,...
8006,ÁUREA,-27.695009
8007,ÂNGULO,-23.195024
8008,ÉRICO CARDOSO,-13.393047
8009,ÓBIDOS,-1.907183


In [15]:
#fusion des dataframes 
geoloc_city=geoloc_city.merge(geo_long, how='left')
geoloc_city=geoloc_city.merge(geo_lat, how='left')
geoloc_city

,geolocation_city,geolocation_lng,geolocation_lat
0,SAO PAULO,-46.633242,-23.570860
1,SÃO PAULO,-46.637556,-23.573388
2,SAO BERNARDO DO CAMPO,-46.563869,-23.707626
3,JUNDIAÍ,-46.900974,-23.184938
4,TABOÃO DA SERRA,-46.779851,-23.620087
...,...,...,...
8006,IBIAÇÁ,-51.856835,-28.056610
8007,SANTA CECILIA DO SUL,-51.908569,-28.171199
8008,CIRÍACO,-51.876023,-28.342325
8009,ESTAÇÃO,-52.255232,-27.916710


On va maintenant récupérer la localisation de chaques clients :

In [16]:
cust = customers[['customer_id','customer_unique_id','customer_city']]
cust['customer_city'] = cust['customer_city'].str.upper()
cust.head()

<ipython-input-16-d72be0300441>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cust['customer_city'] = cust['customer_city'].str.upper()


,customer_id,customer_unique_id,customer_city
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,FRANCA
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,SAO BERNARDO DO CAMPO
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,SAO PAULO
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,MOGI DAS CRUZES
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,CAMPINAS


In [17]:
cust_geo = cust.merge(geoloc_city, left_on='customer_city', right_on='geolocation_city', how='left')
cust_geo = cust_geo.drop(columns=['customer_id','geolocation_city'])
cust_geo = cust_geo.drop_duplicates(keep='first')
cust_geo

,customer_unique_id,customer_city,geolocation_lng,geolocation_lat
0,861eff4711a542e4b93843c6dd7febb0,FRANCA,-47.398437,-20.534057
1,290c77bc529b7ac935b93aa66c333dc3,SAO BERNARDO DO CAMPO,-46.563869,-23.707626
2,060e732b5b29e8181a18229c7b0b2b5e,SAO PAULO,-46.633242,-23.570860
3,259dac757896d24d7702b9acbbff3f3c,MOGI DAS CRUZES,-46.193888,-23.530402
4,345ecd01c38d18a9036ed96c73b8d066,CAMPINAS,-47.068536,-22.900861
...,...,...,...,...
99436,1a29b476fee25c95fbafc67c5ac95cf8,SAO PAULO,-46.633242,-23.570860
99437,d52a67c98be1cf6a5c84435bd38d095d,TABOAO DA SERRA,-46.780156,-23.620738
99438,e9f50caf99f032f0bf3c55141f019d99,FORTALEZA,-38.532430,-3.763108
99439,73c2643a0a458b49f58cea58833b192e,CANOAS,-51.174495,-29.921656


In [18]:
moda(cust_geo.reset_index(drop=True),'customer_unique_id')[moda(cust_geo.reset_index(drop=True),'customer_unique_id')['n']>1]

,customer_unique_id,n,f
0,d44ccec15f5f86d14d6a2cfa67da1975,3,0.003118
1,b17ed96ffdf66518b4cf65e328298e7b,2,0.002079
2,cc44d1f72587f44e3af25f68f1db8cd2,2,0.002079
3,70aa9ef908248a2ec9e03b77ce1ff357,2,0.002079
4,f92fd2c87375f957ece022972b16849b,2,0.002079
...,...,...,...
117,9b8180cd4fe75a8acd576d0408ff8bf4,2,0.002079
118,738ffcf1017b584e9d2684b36e07469c,2,0.002079
119,ce41a50f9ff95ef577cb759483fe9165,2,0.002079
120,ec0f5b4fe392d71f361112c1b508b707,2,0.002079


In [19]:
cust_geo.reset_index(drop=True)[cust_geo.reset_index(drop=True)['customer_unique_id']=='d44ccec15f5f86d14d6a2cfa67da1975']

,customer_unique_id,customer_city,geolocation_lng,geolocation_lat
45100,d44ccec15f5f86d14d6a2cfa67da1975,SAO PAULO,-46.633242,-23.570860
56810,d44ccec15f5f86d14d6a2cfa67da1975,ARACATI,-37.763890,-4.566503
81018,d44ccec15f5f86d14d6a2cfa67da1975,NAVEGANTES,-48.657292,-26.875797


On remarques donc que 122 clients ont plusieures villes, sur un total de 96 096 individus. J'ai donc décider de garder la 1ère ville  : 

In [20]:
cust_geo=cust_geo.drop_duplicates(subset=['customer_unique_id'], keep='first')
cust_geo

,customer_unique_id,customer_city,geolocation_lng,geolocation_lat
0,861eff4711a542e4b93843c6dd7febb0,FRANCA,-47.398437,-20.534057
1,290c77bc529b7ac935b93aa66c333dc3,SAO BERNARDO DO CAMPO,-46.563869,-23.707626
2,060e732b5b29e8181a18229c7b0b2b5e,SAO PAULO,-46.633242,-23.570860
3,259dac757896d24d7702b9acbbff3f3c,MOGI DAS CRUZES,-46.193888,-23.530402
4,345ecd01c38d18a9036ed96c73b8d066,CAMPINAS,-47.068536,-22.900861
...,...,...,...,...
99436,1a29b476fee25c95fbafc67c5ac95cf8,SAO PAULO,-46.633242,-23.570860
99437,d52a67c98be1cf6a5c84435bd38d095d,TABOAO DA SERRA,-46.780156,-23.620738
99438,e9f50caf99f032f0bf3c55141f019d99,FORTALEZA,-38.532430,-3.763108
99439,73c2643a0a458b49f58cea58833b192e,CANOAS,-51.174495,-29.921656


In [21]:
#fusion avec fichier de travail
work=work.merge(cust_geo[['customer_unique_id','geolocation_lng','geolocation_lat']], how='left')
work

,customer_unique_id,geolocation_lng,geolocation_lat
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861
...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656


### B.1.b -  Nombre total de commandes effectuées (nb_orders) : <a name=B.1.b ></a>

In [22]:
#jointure pour récupérer le customer_unique_id de chaques commandes
orders_cust = orders.merge(customers[['customer_unique_id', 'customer_id']], how='left')
orders_cust

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,7c142cf63193a1473d2e66489a9ae977
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,72632f0f9dd73dfee390c9b22eb56dd6
...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,6359f309b166b0196dbf7ad2ac62bb5a
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,da62f9e57a76d978d02ab5362c509660
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,737520a9aad80b3fbbdad19b66b37b30
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,5097a5312c8b157bb7be58ae360ef43c


In [23]:
#on filtre le dataframe précédent afin de ne garder que les commandes effectuées pendant la période considérée 
orders_cust_period=orders_cust[(orders_cust['order_purchase_timestamp']>date_start) & (orders_cust['order_purchase_timestamp']<date_end)]
orders_cust_period

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,7c142cf63193a1473d2e66489a9ae977
5,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09 21:57:05,2017-07-09 22:10:13,2017-07-11 14:58:04,2017-07-26 10:57:55,2017-08-01,80bb27c7c16e8f973207a5086ab329e2
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaT,NaT,2017-05-09,36edbb3fb164b1f16485364b6fb04c73
7,6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16 13:10:30,2017-05-16 13:22:11,2017-05-22 10:07:46,2017-05-26 12:55:51,2017-06-07,932afa1e708222e5821dac9cd5db4cae
...,...,...,...,...,...,...,...,...,...
99433,9115830be804184b91f5c00f6f49f92d,da2124f134f5dfbce9d06f29bdb6c308,delivered,2017-10-04 19:57:37,2017-10-04 20:07:14,2017-10-05 16:52:52,2017-10-20 20:25:45,2017-11-07,c716cf2b5b86fb24257cffe9e7969df8
99435,880675dff2150932f1601e1c07eadeeb,47cd45a6ac7b9fb16537df2ccffeb5ac,delivered,2017-02-23 09:05:12,2017-02-23 09:15:11,2017-03-01 10:22:52,2017-03-06 11:08:08,2017-03-22,831ce3f1bacbd424fc4e38fbd4d66d29
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,6359f309b166b0196dbf7ad2ac62bb5a
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,737520a9aad80b3fbbdad19b66b37b30


In [24]:
#on calcule le nombre de commandes, order_id, pour chaques client unique, customer_unique_id
nb_orders = pd.DataFrame(orders_cust_period.groupby('customer_unique_id')['order_id'].count()).reset_index()
nb_orders = nb_orders.rename(columns={"order_id": "nb_orders"})
nb_orders

,customer_unique_id,nb_orders
0,0000f46a3911fa3c0805444483337064,1
1,0000f6ccb0745a6a4b88665a16c9f078,1
2,0004aac84e0df4da2b147fca70cf8255,1
3,0005e1862207bf6ccc02e4228effd9a0,1
4,0006fdc98a402fceb4eb0ee528f6a8d4,1
...,...,...
49744,fffbf87b7a1a6fa8b03f081c5f51a201,1
49745,fffcf5a5ff07b0908bd4e2dbc735a684,1
49746,fffea47cd6d3cc0a88bd621562a9d061,1
49747,ffff371b4d645b6ecea244b27531430a,1


In [25]:
#on jointe avec le fichier final
work=work.merge(nb_orders, how='left')
#on valorise les données manquantes par 0.0
work['nb_orders']=work['nb_orders'].fillna(0.0) 
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0
...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0


### B.1.c - Note de satisfaction moyenne (mean_review_score) : <a name=B.1.c ></a>

In [26]:
recap(order_reviews)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
count,100000,100000,100000,11715,41753,100000,100000
unique,99173,99441,NaN,4600,36921,637,99010
top,3415c9f764e478409e8e0660ae816dd2,df56136b8031ecd28e200bb18e6ddb2e,NaN,Recomendo,Muito bom,2017-12-19 00:00:00,2017-06-15 23:21:05
freq,3,3,NaN,426,230,466,4
mean,NaN,NaN,4.07089,NaN,NaN,NaN,NaN
std,NaN,NaN,1.35966,NaN,NaN,NaN,NaN
min,NaN,NaN,1,NaN,NaN,NaN,NaN
25%,NaN,NaN,4,NaN,NaN,NaN,NaN
50%,NaN,NaN,5,NaN,NaN,NaN,NaN
75%,NaN,NaN,5,NaN,NaN,NaN,NaN


On retrouve plusieurs occurence de **review_id** et de **order_id**, regardons pourquoi :

In [27]:
order_reviews[order_reviews['order_id']=='c88b1d1b157a9999ce368f218a407141']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
1997,ffb8cff872a625632ac983eb1f88843c,c88b1d1b157a9999ce368f218a407141,3,NaN,NaN,2017-07-22 00:00:00,2017-07-26 13:41:07
83187,202b5f44d09cd3cfc0d6bd12f01b044c,c88b1d1b157a9999ce368f218a407141,5,NaN,NaN,2017-07-22 00:00:00,2017-07-26 13:40:22
90069,fb96ea2ef8cce1c888f4d45c8e22b793,c88b1d1b157a9999ce368f218a407141,5,NaN,NaN,2017-07-21 00:00:00,2017-07-26 13:45:15


Ci dessus, on observe plusieurs **review_id** faisant référence au même **order_id**. Ce qui ne semble pas normal, a part si le client peut donner un avis à chaque produit de sa commande et pas à la commande elle même. 

In [28]:
order_reviews[order_reviews['review_id']=='7b606b0d57b078384f0b58eac1d41d78']

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
7554,7b606b0d57b078384f0b58eac1d41d78,f3028a8f41ea1ee2b461420913663f97,5,NaN,NaN,2017-02-15 00:00:00,2017-02-21 23:30:22
60329,7b606b0d57b078384f0b58eac1d41d78,2deb17060fc1ce18a85eba953ddcdeaf,5,NaN,NaN,2017-02-15 00:00:00,2017-02-21 23:30:22
61553,7b606b0d57b078384f0b58eac1d41d78,2f8f31eb2f7b6572836d662a6625c8e4,5,NaN,NaN,2017-02-15 00:00:00,2017-02-21 23:30:22


Ci dessus, on observe plusieurs **order_id** faisant référence au même **review_id**. Ce qui ne semble pas normal, car cela semble bizarre de pouvoir donner une même avis pour des commandes différentes. <br>
Partant de ces observations j'ai choisi de conserver les doublons de **order_id** en admettant qu'un client peut donner plusieurs avis sur une même commande, par exemple en notant chaques produits indépendement. Par contre, je considère que ce n'est pas possible de donner un avis identique pour plusieurs commandes. On va donc supprimer les doublons sur **review_id** :

In [29]:
reviews = order_reviews.drop_duplicates(subset=['review_id'],keep='first')
reviews

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
...,...,...,...,...,...,...,...
99995,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09 00:00:00,2017-12-11 20:06:42
99996,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43
99997,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,NaN,2018-07-01 00:00:00,2018-07-02 12:59:13
99998,be360f18f5df1e0541061c87021e6d93,f8bd3f2000c28c5342fedeb5e50f2e75,1,NaN,Solicitei a compra de uma capa de retrovisor c...,2017-12-15 00:00:00,2017-12-16 01:29:43


In [30]:
#jointure pour récupérer les customer_id de chaque review
reviews=reviews[['order_id','review_score']].merge(orders[['order_id','customer_id','order_purchase_timestamp']], how='left')
reviews

,order_id,review_score,customer_id,order_purchase_timestamp
0,73fc7af87114b39712e6da79b0a377eb,4,41dcb106f807e993532d446263290104,2018-01-11 15:30:49
1,a548910a1c6147796b98fdf73dbeba33,5,8a2e7ef9053dea531e4dc76bd6d853e6,2018-02-28 12:25:19
2,f9e4b658b201a9f2ecdecbb34bed034b,5,e226dfed6544df5b7b87a48208690feb,2018-02-03 09:56:22
3,658677c97b385a9be170737859d3511b,5,de6dff97e5f1ba84a3cd9a3bc97df5f6,2017-04-09 17:41:13
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,5986b333ca0d44534a156a52a8e33a83,2018-02-10 10:59:03
...,...,...,...,...
99168,22ec9f0669f784db00fa86d035cf8602,5,d0d7086dea6fcf42b9b690b9f3745c58,2017-12-03 21:45:23
99169,55d4004744368f5571d1f590031933e4,5,fcc7b1caafe3b77fd587bab964c4d1fb,2018-03-18 09:52:19
99170,7725825d039fc1f0ceb7635e3f7d9206,4,3aa00401736823c73e9fe8683328fa6b,2018-06-22 16:47:28
99171,f8bd3f2000c28c5342fedeb5e50f2e75,1,8df587ce8a11ee97b3de9ef3405245c2,2017-12-10 18:44:23


In [31]:
#on filtre pour garder uniquement les commandes effectuées dans la période considérée
reviews=reviews[(reviews['order_purchase_timestamp']>date_start) & (reviews['order_purchase_timestamp']<date_end)]
reviews

,order_id,review_score,customer_id,order_purchase_timestamp
0,73fc7af87114b39712e6da79b0a377eb,4,41dcb106f807e993532d446263290104,2018-01-11 15:30:49
3,658677c97b385a9be170737859d3511b,5,de6dff97e5f1ba84a3cd9a3bc97df5f6,2017-04-09 17:41:13
6,e48aa0d2dcec3a2e87348811bcfdf22b,5,840be0730010e9580efd289bc9f37706,2017-06-30 15:38:46
8,9c214ac970e84273583ab523dfafd09b,5,daf813bc2e41c86027d086e8dfb35571,2017-05-08 13:35:48
10,cdf9aa68e72324eeb25c7de974696ee2,5,272b46feddddc51904608ddd90feee33,2017-12-13 14:59:45
...,...,...,...,...
99160,f62f01ec64c6b221f926ba560907d8f7,3,a35ef57cd99a99e77e58316ab0cc2927,2017-02-14 20:55:49
99165,48f7ee67313eda32bfcf5b9c1dd9522d,5,3d7697c0971cb2606444adae322d8bb3,2017-11-24 09:31:02
99168,22ec9f0669f784db00fa86d035cf8602,5,d0d7086dea6fcf42b9b690b9f3745c58,2017-12-03 21:45:23
99171,f8bd3f2000c28c5342fedeb5e50f2e75,1,8df587ce8a11ee97b3de9ef3405245c2,2017-12-10 18:44:23


In [32]:
#retrait de colonnes qui ne nous intérressent pas
reviews=reviews.drop(columns=['order_purchase_timestamp','order_id'])
reviews

,review_score,customer_id
0,4,41dcb106f807e993532d446263290104
3,5,de6dff97e5f1ba84a3cd9a3bc97df5f6
6,5,840be0730010e9580efd289bc9f37706
8,5,daf813bc2e41c86027d086e8dfb35571
10,5,272b46feddddc51904608ddd90feee33
...,...,...
99160,3,a35ef57cd99a99e77e58316ab0cc2927
99165,5,3d7697c0971cb2606444adae322d8bb3
99168,5,d0d7086dea6fcf42b9b690b9f3745c58
99171,1,8df587ce8a11ee97b3de9ef3405245c2


In [33]:
#jointure pour récupérer le customer_unique_id
reviews=reviews.merge(customers[['customer_id','customer_unique_id']], how='left')
reviews

,review_score,customer_id,customer_unique_id
0,4,41dcb106f807e993532d446263290104,68a5590b9926689be4e10f4ae2db21a8
1,5,de6dff97e5f1ba84a3cd9a3bc97df5f6,c8cf6cb6b838dc7a33ed199b825e8616
2,5,840be0730010e9580efd289bc9f37706,dcd4940b3f96a3e2b73d8f73387230cf
3,5,daf813bc2e41c86027d086e8dfb35571,8c89391790076834500661cc1e5d6860
4,5,272b46feddddc51904608ddd90feee33,485fa1b8db05ed6fa0062ece4ce80d2e
...,...,...,...
51164,3,a35ef57cd99a99e77e58316ab0cc2927,d3e09f23a2bed5fa4eef73fb5fd68cfc
51165,5,3d7697c0971cb2606444adae322d8bb3,f4513d9cd4f25a8254176ceda0abdc5e
51166,5,d0d7086dea6fcf42b9b690b9f3745c58,597cbb334f18a671472f7e16648228b4
51167,1,8df587ce8a11ee97b3de9ef3405245c2,c3793040f54f0f511e5bcaa8937d0b0a


In [34]:
#retrait du customer_id
reviews=reviews.drop(columns=['customer_id'])
reviews

,review_score,customer_unique_id
0,4,68a5590b9926689be4e10f4ae2db21a8
1,5,c8cf6cb6b838dc7a33ed199b825e8616
2,5,dcd4940b3f96a3e2b73d8f73387230cf
3,5,8c89391790076834500661cc1e5d6860
4,5,485fa1b8db05ed6fa0062ece4ce80d2e
...,...,...
51164,3,d3e09f23a2bed5fa4eef73fb5fd68cfc
51165,5,f4513d9cd4f25a8254176ceda0abdc5e
51166,5,597cbb334f18a671472f7e16648228b4
51167,1,c3793040f54f0f511e5bcaa8937d0b0a


In [35]:
#calcul de la moyenne des notes pour chaques customer_unique_id
mean_reviews=pd.DataFrame(reviews.groupby('customer_unique_id')['review_score'].mean())
mean_reviews=mean_reviews.reset_index()
mean_reviews=mean_reviews.rename(columns={"review_score": "mean_review_score"})
mean_reviews

,customer_unique_id,mean_review_score
0,0000f46a3911fa3c0805444483337064,3.0
1,0000f6ccb0745a6a4b88665a16c9f078,4.0
2,0004aac84e0df4da2b147fca70cf8255,5.0
3,0005e1862207bf6ccc02e4228effd9a0,4.0
4,0006fdc98a402fceb4eb0ee528f6a8d4,3.0
...,...,...
49744,fffbf87b7a1a6fa8b03f081c5f51a201,5.0
49745,fffcf5a5ff07b0908bd4e2dbc735a684,5.0
49746,fffea47cd6d3cc0a88bd621562a9d061,4.0
49747,ffff371b4d645b6ecea244b27531430a,5.0


In [36]:
#jointure avec le fichier de travail
work=work.merge(mean_reviews, how='left')
work['mean_review_score']=work['mean_review_score'].fillna(-1.0) #valorisation données manquantes pas -1.0
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0
...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0


### B.1.d - Moyen de paiement préféré (hors vouchers/bons d'achats) : <a name=B.1.d ></a>
On va chercher à récupérer le moyen de paiment le plus utilisé par les clients, hors 'voucher'/bons d'achats, en effet on ne considère pas le bon d'achat comme moyen de paiment en soit mais plutot comme complément au moyen de paiement. On comptera l'utilisation des bons d'achats séparément.

In [37]:
pay_type = order_payments[order_payments['payment_type']!='voucher']
pay_type

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


In [38]:
#jointure pour récupérer le customer_id
pay_type=pay_type[['order_id','payment_type']].merge(orders[['order_id','customer_id','order_purchase_timestamp']], how='left')
pay_type

,order_id,payment_type,customer_id,order_purchase_timestamp
0,b81ef226f3fe1789b1e8b2acac839d17,credit_card,0a8556ac6be836b46b3e89920d59291c,2018-04-25 22:01:49
1,a9810da82917af2d9aefd1278f1dcfa0,credit_card,f2c7fc58a9de810828715166c672f10a,2018-06-26 11:01:38
2,25e8ea4e93396b6fa0d3dd708e76c1bd,credit_card,25b14b69de0b6e184ae6fe2755e478f9,2017-12-12 11:19:55
3,ba78997921bbcdc1373bb41e913ab953,credit_card,7a5d8efaaa1081f800628c30d2b0728f,2017-12-06 12:04:06
4,42fdf880ba16b47b59251dd489d4441a,credit_card,15fd6fb8f8312dbb4674e4518d6fa3b3,2018-05-21 13:59:17
...,...,...,...,...
98106,0406037ad97740d563a178ecc7a2075c,boleto,5d576cb2dfa3bc05612c392a1ee9c654,2018-03-08 16:57:23
98107,7b905861d7c825891d6347454ea7863f,credit_card,2079230c765a88530822a34a4cec2aa0,2017-08-18 09:45:35
98108,32609bbb3dd69b3c066a6860554a77bf,credit_card,e4abb5057ec8cfda9759c0dc415a8188,2017-11-18 17:27:14
98109,b8b61059626efa996a60be9bb9320e10,credit_card,5d719b0d300663188169c6560e243f27,2018-08-07 23:26:13


In [39]:
#on filtrer pour garder uniquement les commandes effectuées pendant la période considérée 
pay_type=pay_type[(pay_type['order_purchase_timestamp']>date_start) & (pay_type['order_purchase_timestamp']<date_end)]
pay_type

,order_id,payment_type,customer_id,order_purchase_timestamp
2,25e8ea4e93396b6fa0d3dd708e76c1bd,credit_card,25b14b69de0b6e184ae6fe2755e478f9,2017-12-12 11:19:55
3,ba78997921bbcdc1373bb41e913ab953,credit_card,7a5d8efaaa1081f800628c30d2b0728f,2017-12-06 12:04:06
6,771ee386b001f06208a7419e4fc1bbd7,credit_card,206c0fcc717333d169ff95933fb47341,2017-06-23 13:11:27
7,3d7239c394a212faae122962df514ac7,credit_card,4d7a9b8bba459dce09d1b8fa637a0cba,2017-06-05 10:45:54
9,0573b5e23cbd798006520e1d5b4c6714,boleto,c4e17eff78176dfe3401f03db0346f85,2017-07-04 20:32:10
...,...,...,...,...
98101,0872d0faafe0cb56e6f2c594cca1522c,credit_card,dd2f0d64271438f6ad3a75d2a930a4b5,2017-09-26 12:10:26
98103,2e3d96522c8c378d4135c12a65c977aa,credit_card,cd0a090974c3b64acf613c18f9fcfe83,2017-11-23 17:11:23
98107,7b905861d7c825891d6347454ea7863f,credit_card,2079230c765a88530822a34a4cec2aa0,2017-08-18 09:45:35
98108,32609bbb3dd69b3c066a6860554a77bf,credit_card,e4abb5057ec8cfda9759c0dc415a8188,2017-11-18 17:27:14


In [40]:
#jointure pour récupérer les customer_unique_id
pay_type=pay_type.drop(columns=['order_purchase_timestamp','order_id'])
pay_type=pay_type.merge(customers[['customer_id','customer_unique_id']], how='left')
pay_type

,payment_type,customer_id,customer_unique_id
0,credit_card,25b14b69de0b6e184ae6fe2755e478f9,6f70c0b2f7552832ba46eb57b1c5651e
1,credit_card,7a5d8efaaa1081f800628c30d2b0728f,87695ed086ebd36f20404c82d20fca87
2,credit_card,206c0fcc717333d169ff95933fb47341,9a3526eee723accd014ec683afe5bf20
3,credit_card,4d7a9b8bba459dce09d1b8fa637a0cba,9db67268a3fee1d4b13faaeb90af07c0
4,boleto,c4e17eff78176dfe3401f03db0346f85,511c345b0383af6ef25406c564e4d6b6
...,...,...,...
50602,credit_card,dd2f0d64271438f6ad3a75d2a930a4b5,4a57b2a3676de278e26a50b7e60214e6
50603,credit_card,cd0a090974c3b64acf613c18f9fcfe83,1c157ec47b1201b33c6ea9cc61c966fb
50604,credit_card,2079230c765a88530822a34a4cec2aa0,53b30ca78efb2b7efcd3f9e461587eb2
50605,credit_card,e4abb5057ec8cfda9759c0dc415a8188,d3c7da954a324253814096bcaf240e4e


On récupère le 'payment_type' le plus fréquent pour chaques customer_unique_id :
- 1: calcul de la frequence des payment_type par customer_unique_id

In [41]:
most_freq_pay_type=pd.DataFrame(pay_type.groupby('customer_unique_id')['payment_type'].value_counts())
most_freq_pay_type = most_freq_pay_type.rename(columns={"payment_type": "freq_payment_type"})
most_freq_pay_type=most_freq_pay_type.reset_index()
most_freq_pay_type

,customer_unique_id,payment_type,freq_payment_type
0,0000f46a3911fa3c0805444483337064,credit_card,1
1,0000f6ccb0745a6a4b88665a16c9f078,credit_card,1
2,0004aac84e0df4da2b147fca70cf8255,credit_card,1
3,0005e1862207bf6ccc02e4228effd9a0,credit_card,1
4,0006fdc98a402fceb4eb0ee528f6a8d4,credit_card,1
...,...,...,...
49035,fffbf87b7a1a6fa8b03f081c5f51a201,credit_card,1
49036,fffcf5a5ff07b0908bd4e2dbc735a684,credit_card,1
49037,fffea47cd6d3cc0a88bd621562a9d061,credit_card,1
49038,ffff371b4d645b6ecea244b27531430a,credit_card,1


- 2: on classe suivant freq_payment_type 

In [42]:
most_freq_pay_type=most_freq_pay_type.sort_values(by='freq_payment_type', ascending=False)
most_freq_pay_type

,customer_unique_id,payment_type,freq_payment_type
26987,8d50f5eadf50201ccdcedfb9e2ac8455,credit_card,10
19245,6469f99c1f9dfae7733b25662e7f1782,credit_card,6
42709,de34b16117594161a6a89c50b289d35a,credit_card,6
13764,47c1a3033b8b77b3ab6e109eb4d5fdf3,boleto,6
11930,3e43e6105506432c953e165fb2acf44c,credit_card,5
...,...,...,...
16563,5642d883f45725581f344df11906a2c9,credit_card,1
16564,5643c90bcfe88c649de3f7f959b5e877,credit_card,1
16565,564935b7af878f03c1b137d326c005fd,credit_card,1
16566,564988508a81dca36512b04a1ce906f9,credit_card,1


- 3: on supprime les doublons sur customer_unique_id 

In [43]:
most_freq_pay_type=most_freq_pay_type.drop_duplicates(subset=['customer_unique_id'],keep='first')
most_freq_pay_type

,customer_unique_id,payment_type,freq_payment_type
26987,8d50f5eadf50201ccdcedfb9e2ac8455,credit_card,10
19245,6469f99c1f9dfae7733b25662e7f1782,credit_card,6
42709,de34b16117594161a6a89c50b289d35a,credit_card,6
13764,47c1a3033b8b77b3ab6e109eb4d5fdf3,boleto,6
11930,3e43e6105506432c953e165fb2acf44c,credit_card,5
...,...,...,...
16563,5642d883f45725581f344df11906a2c9,credit_card,1
16564,5643c90bcfe88c649de3f7f959b5e877,credit_card,1
16565,564935b7af878f03c1b137d326c005fd,credit_card,1
16566,564988508a81dca36512b04a1ce906f9,credit_card,1


In [44]:
#jointure avec le fichier de travail
work=work.merge(most_freq_pay_type[['customer_unique_id','payment_type']], how='left')
work['payment_type']=work['payment_type'].fillna('missing')
work = work.rename(columns={"payment_type": "most_freq_payment_type"})
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing
...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card


### B.1.e - Nombre de bon d'achats utilisés (nb_voucher) : <a name=B.1.e ></a>

In [45]:
#on récupère les paiements faisant référence à un bon d'achat
voucher=order_payments[order_payments['payment_type']=='voucher']
voucher

,order_id,payment_sequential,payment_type,payment_installments,payment_value
25,5cfd514482e22bc992e7693f0e3e8df7,2,voucher,1,45.17
68,947e9c6bfd83a06953c420ef92d3da6e,1,voucher,1,69.46
70,e3e9e3295f4e1512167c232a0aaae8a5,1,voucher,1,50.80
75,3689194c14ad4e2e7361ebd1df0e77b0,2,voucher,1,57.53
102,21b8b46679ea6482cbf911d960490048,2,voucher,1,43.12
...,...,...,...,...,...
103747,cf101c3abd3c061ca9f78c1bbb1125af,3,voucher,1,8.46
103750,d0bff47153ef056bb4f884a2ec2f0691,2,voucher,1,41.89
103795,59de68e3ef040153cc9ea7978eaec149,1,voucher,1,65.71
103798,4f259869d5c650af6f847b027032515b,1,voucher,1,176.56


In [46]:
#jointure pour récupèrer le customer_id & le order_purchase_timestamp
voucher=voucher[['order_id','payment_value']].merge(orders[['order_id','customer_id','order_purchase_timestamp']],how='left')
voucher

,order_id,payment_value,customer_id,order_purchase_timestamp
0,5cfd514482e22bc992e7693f0e3e8df7,45.17,519a8af813fe88578029697625439e8b,2017-10-13 17:19:17
1,947e9c6bfd83a06953c420ef92d3da6e,69.46,2737211835d5ea370af15ee145f7840c,2018-08-15 15:13:32
2,e3e9e3295f4e1512167c232a0aaae8a5,50.80,76382f6496effe36685eb200ef95b3ac,2017-02-25 00:02:22
3,3689194c14ad4e2e7361ebd1df0e77b0,57.53,40e2d0d7f6579118d9c02dcabf1fff65,2017-10-18 16:01:53
4,21b8b46679ea6482cbf911d960490048,43.12,1e7c63005e58c8c02c20ef58f76735a0,2018-05-08 14:23:15
...,...,...,...,...
5770,cf101c3abd3c061ca9f78c1bbb1125af,8.46,351e40989da90e70487765f6ea15d54b,2017-07-13 10:42:37
5771,d0bff47153ef056bb4f884a2ec2f0691,41.89,5bbd3f38fe696e4bc73a9f914a6716e6,2017-11-23 15:05:54
5772,59de68e3ef040153cc9ea7978eaec149,65.71,267ab9426bea353f82ef7c5406d947c3,2017-10-12 23:26:41
5773,4f259869d5c650af6f847b027032515b,176.56,6fc16fa677c8528424093b92165f7ddb,2018-04-28 19:25:30


In [47]:
#filtrer pour conserver uniquement les commandes effectuées dans la période considérée
voucher=voucher[(voucher['order_purchase_timestamp']>date_start) & (voucher['order_purchase_timestamp']<date_end)]
voucher

,order_id,payment_value,customer_id,order_purchase_timestamp
0,5cfd514482e22bc992e7693f0e3e8df7,45.17,519a8af813fe88578029697625439e8b,2017-10-13 17:19:17
2,e3e9e3295f4e1512167c232a0aaae8a5,50.80,76382f6496effe36685eb200ef95b3ac,2017-02-25 00:02:22
3,3689194c14ad4e2e7361ebd1df0e77b0,57.53,40e2d0d7f6579118d9c02dcabf1fff65,2017-10-18 16:01:53
6,e2cd740333fe4585ead5c50c61edb6a5,163.25,4a51cb396dec2a0c8395363ebae19916,2018-01-08 21:24:09
8,487c1451b8fd7347d0e80e5aca887e91,30.00,ca454a420355c83f95fe93083dc31c81,2017-03-21 08:50:00
...,...,...,...,...
5769,4fbc2bd17fd14bf9e9b4f28f96351edf,27.86,948f067df4cf5b2c51054d8a7603aaca,2017-03-14 19:26:29
5770,cf101c3abd3c061ca9f78c1bbb1125af,8.46,351e40989da90e70487765f6ea15d54b,2017-07-13 10:42:37
5771,d0bff47153ef056bb4f884a2ec2f0691,41.89,5bbd3f38fe696e4bc73a9f914a6716e6,2017-11-23 15:05:54
5772,59de68e3ef040153cc9ea7978eaec149,65.71,267ab9426bea353f82ef7c5406d947c3,2017-10-12 23:26:41


In [48]:
#jointure pour récupérer le customer_unique_id

#on retire la colonne order_purchase_timestamp
voucher = voucher.drop(columns=['order_purchase_timestamp'])
voucher= voucher.merge(customers[['customer_id','customer_unique_id']],how='left')
voucher

,order_id,payment_value,customer_id,customer_unique_id
0,5cfd514482e22bc992e7693f0e3e8df7,45.17,519a8af813fe88578029697625439e8b,e2381d9bb9b49c35fff077d4ae613816
1,e3e9e3295f4e1512167c232a0aaae8a5,50.80,76382f6496effe36685eb200ef95b3ac,0b11937f7f94a269cc94a3378be84839
2,3689194c14ad4e2e7361ebd1df0e77b0,57.53,40e2d0d7f6579118d9c02dcabf1fff65,764bf13c9f7df277b9c0950b88b4c3eb
3,e2cd740333fe4585ead5c50c61edb6a5,163.25,4a51cb396dec2a0c8395363ebae19916,831d5ed354fc1d51fa7e6a5f981e8069
4,487c1451b8fd7347d0e80e5aca887e91,30.00,ca454a420355c83f95fe93083dc31c81,12456b22f2fe7567211f0f29dc7dc58e
...,...,...,...,...
3363,4fbc2bd17fd14bf9e9b4f28f96351edf,27.86,948f067df4cf5b2c51054d8a7603aaca,74b8735c819aa3ddb7d86d0ea39efe51
3364,cf101c3abd3c061ca9f78c1bbb1125af,8.46,351e40989da90e70487765f6ea15d54b,74adf920dbd3d2e6e9f18acd88b2fd41
3365,d0bff47153ef056bb4f884a2ec2f0691,41.89,5bbd3f38fe696e4bc73a9f914a6716e6,7305430719d715992b00be82af4a6aa8
3366,59de68e3ef040153cc9ea7978eaec149,65.71,267ab9426bea353f82ef7c5406d947c3,b117d1f1ebf30b6c7ebd5ee68505e43c


In [49]:
#calcul du nombre de bons d'achats utilisés par client unique
nb_voucher=pd.DataFrame(voucher.groupby('customer_unique_id')['customer_id'].count()).reset_index()
nb_voucher=nb_voucher.rename(columns={"customer_id": "nb_voucher"})
nb_voucher

,customer_unique_id,nb_voucher
0,0028a7d8db7b0247652509358ad8d755,1
1,00324c9f4d710e7bac5c5ba679714430,1
2,00986cafefb6e670cceca01694bda599,1
3,00b309d0151d92709de70e43d662711b,2
4,00bd169997f142571c113645554663d6,1
...,...,...
2122,ff70aa1b53f643263ead50eed3166164,3
2123,ff9bc66daa6702543b16f3c4897e300b,1
2124,ffbb866d7c0d272f9fe12de1b9ee9173,1
2125,ffc04aafbf624bd130516cc1d195626a,1


In [50]:
#jointure avec le fichier de travail 
work=work.merge(nb_voucher, how='left')
work['nb_voucher']=work['nb_voucher'].fillna(0)
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0
...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0


### B.1.f - Montant total des bon d'achats utilisés (voucher_value) : <a name=B.1.f ></a>

In [51]:
#calcul du montant total des bons d'achats utilisés par client unique
voucher_value=pd.DataFrame(voucher.groupby('customer_unique_id')['payment_value'].sum()).reset_index()
voucher_value=voucher_value.rename(columns={"payment_value": "voucher_value"})
voucher_value

,customer_unique_id,voucher_value
0,0028a7d8db7b0247652509358ad8d755,293.54
1,00324c9f4d710e7bac5c5ba679714430,31.73
2,00986cafefb6e670cceca01694bda599,43.92
3,00b309d0151d92709de70e43d662711b,107.78
4,00bd169997f142571c113645554663d6,9.94
...,...,...
2122,ff70aa1b53f643263ead50eed3166164,47.22
2123,ff9bc66daa6702543b16f3c4897e300b,75.02
2124,ffbb866d7c0d272f9fe12de1b9ee9173,81.96
2125,ffc04aafbf624bd130516cc1d195626a,17.78


In [52]:
#jointure avec le fichier de travail
work=work.merge(voucher_value,how='left')
work['voucher_value']=work['voucher_value'].fillna(0.0)
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0
...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0


### B.1.g - Nombre de versements moyen (mean_payment_installments) : <a name=B.1.g ></a>
Sur Olist il est possible d'effectuer des paiements en plusieurs fois, on va donc voir les habitudes de chaques clients sur les paiements en plusieurs fois.

In [53]:
#jointure pour lier les informations des commandes et le nombre de paiements
pay_inst=order_payments[['order_id','payment_installments']].merge(orders[['order_id','customer_id','order_purchase_timestamp']], how='left')
pay_inst

,order_id,payment_installments,customer_id,order_purchase_timestamp
0,b81ef226f3fe1789b1e8b2acac839d17,8,0a8556ac6be836b46b3e89920d59291c,2018-04-25 22:01:49
1,a9810da82917af2d9aefd1278f1dcfa0,1,f2c7fc58a9de810828715166c672f10a,2018-06-26 11:01:38
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,25b14b69de0b6e184ae6fe2755e478f9,2017-12-12 11:19:55
3,ba78997921bbcdc1373bb41e913ab953,8,7a5d8efaaa1081f800628c30d2b0728f,2017-12-06 12:04:06
4,42fdf880ba16b47b59251dd489d4441a,2,15fd6fb8f8312dbb4674e4518d6fa3b3,2018-05-21 13:59:17
...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,1,5d576cb2dfa3bc05612c392a1ee9c654,2018-03-08 16:57:23
103882,7b905861d7c825891d6347454ea7863f,2,2079230c765a88530822a34a4cec2aa0,2017-08-18 09:45:35
103883,32609bbb3dd69b3c066a6860554a77bf,1,e4abb5057ec8cfda9759c0dc415a8188,2017-11-18 17:27:14
103884,b8b61059626efa996a60be9bb9320e10,5,5d719b0d300663188169c6560e243f27,2018-08-07 23:26:13


In [54]:
#filtre pour récupérer uniquement les commandes effectuées dans la période considérée
pay_inst=pay_inst[(pay_inst['order_purchase_timestamp']>date_start) & (pay_inst['order_purchase_timestamp']<date_end)]
pay_inst

,order_id,payment_installments,customer_id,order_purchase_timestamp
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,25b14b69de0b6e184ae6fe2755e478f9,2017-12-12 11:19:55
3,ba78997921bbcdc1373bb41e913ab953,8,7a5d8efaaa1081f800628c30d2b0728f,2017-12-06 12:04:06
6,771ee386b001f06208a7419e4fc1bbd7,1,206c0fcc717333d169ff95933fb47341,2017-06-23 13:11:27
7,3d7239c394a212faae122962df514ac7,3,4d7a9b8bba459dce09d1b8fa637a0cba,2017-06-05 10:45:54
9,0573b5e23cbd798006520e1d5b4c6714,1,c4e17eff78176dfe3401f03db0346f85,2017-07-04 20:32:10
...,...,...,...,...
103876,0872d0faafe0cb56e6f2c594cca1522c,2,dd2f0d64271438f6ad3a75d2a930a4b5,2017-09-26 12:10:26
103878,2e3d96522c8c378d4135c12a65c977aa,4,cd0a090974c3b64acf613c18f9fcfe83,2017-11-23 17:11:23
103882,7b905861d7c825891d6347454ea7863f,2,2079230c765a88530822a34a4cec2aa0,2017-08-18 09:45:35
103883,32609bbb3dd69b3c066a6860554a77bf,1,e4abb5057ec8cfda9759c0dc415a8188,2017-11-18 17:27:14


In [55]:
#jointure pour récupérer le customer_unique_id

#on retire la colonne order_purchase_timestamp
pay_inst=pay_inst.drop(columns=['order_purchase_timestamp']) 
pay_inst=pay_inst.merge(customers[['customer_id','customer_unique_id']], how='left')
pay_inst

,order_id,payment_installments,customer_id,customer_unique_id
0,25e8ea4e93396b6fa0d3dd708e76c1bd,1,25b14b69de0b6e184ae6fe2755e478f9,6f70c0b2f7552832ba46eb57b1c5651e
1,ba78997921bbcdc1373bb41e913ab953,8,7a5d8efaaa1081f800628c30d2b0728f,87695ed086ebd36f20404c82d20fca87
2,771ee386b001f06208a7419e4fc1bbd7,1,206c0fcc717333d169ff95933fb47341,9a3526eee723accd014ec683afe5bf20
3,3d7239c394a212faae122962df514ac7,3,4d7a9b8bba459dce09d1b8fa637a0cba,9db67268a3fee1d4b13faaeb90af07c0
4,0573b5e23cbd798006520e1d5b4c6714,1,c4e17eff78176dfe3401f03db0346f85,511c345b0383af6ef25406c564e4d6b6
...,...,...,...,...
53970,0872d0faafe0cb56e6f2c594cca1522c,2,dd2f0d64271438f6ad3a75d2a930a4b5,4a57b2a3676de278e26a50b7e60214e6
53971,2e3d96522c8c378d4135c12a65c977aa,4,cd0a090974c3b64acf613c18f9fcfe83,1c157ec47b1201b33c6ea9cc61c966fb
53972,7b905861d7c825891d6347454ea7863f,2,2079230c765a88530822a34a4cec2aa0,53b30ca78efb2b7efcd3f9e461587eb2
53973,32609bbb3dd69b3c066a6860554a77bf,1,e4abb5057ec8cfda9759c0dc415a8188,d3c7da954a324253814096bcaf240e4e


In [56]:
#on calcule la moyenne de payment_installments pour chaques client unique
pay_inst_mean=pd.DataFrame(pay_inst.groupby('customer_unique_id')['payment_installments'].mean()).reset_index()
pay_inst_mean=pay_inst_mean.rename(columns={"payment_installments": "mean_payment_installments"})
pay_inst_mean

,customer_unique_id,mean_payment_installments
0,0000f46a3911fa3c0805444483337064,8.0
1,0000f6ccb0745a6a4b88665a16c9f078,4.0
2,0004aac84e0df4da2b147fca70cf8255,6.0
3,0005e1862207bf6ccc02e4228effd9a0,3.0
4,0006fdc98a402fceb4eb0ee528f6a8d4,2.0
...,...,...
49744,fffbf87b7a1a6fa8b03f081c5f51a201,2.0
49745,fffcf5a5ff07b0908bd4e2dbc735a684,10.0
49746,fffea47cd6d3cc0a88bd621562a9d061,1.0
49747,ffff371b4d645b6ecea244b27531430a,1.0


In [57]:
#jointure avec le fichier de travail
work=work.merge(pay_inst_mean, how='left')
work['mean_payment_installments']=work['mean_payment_installments'].fillna(0.0)
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value,mean_payment_installments
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0,2.0
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0,8.0
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0,0.0
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0,0.0
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0,0.0
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0,2.0


## B.2 - Variables "RFM" : <a name=B.2 ></a>
On passe maintenant à la création de variables en lien avec la segmentation "RFM". Ce sont donc des variables qui mettent en évidence des habitudes de consommation.
### B.2.a - Récence : Nombre de jours depuis la dernière commande (nb_days_last_order) <a name=B.2.a ></a>

In [58]:
#récupération de toutes les commandes effectuées sur la période considérée
ords=orders[(orders['order_purchase_timestamp']>date_start) & (orders['order_purchase_timestamp']<date_end)]
ords

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
5,a4591c265e18cb1dcee52889e2d8acc3,503740e9ca751ccdda7ba28e9ab8f608,delivered,2017-07-09 21:57:05,2017-07-09 22:10:13,2017-07-11 14:58:04,2017-07-26 10:57:55,2017-08-01
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaT,NaT,2017-05-09
7,6514b8ad8028c9f2cc2374ded245783f,9bdf08b4b3b52b5526ff42d37d47f222,delivered,2017-05-16 13:10:30,2017-05-16 13:22:11,2017-05-22 10:07:46,2017-05-26 12:55:51,2017-06-07
...,...,...,...,...,...,...,...,...
99433,9115830be804184b91f5c00f6f49f92d,da2124f134f5dfbce9d06f29bdb6c308,delivered,2017-10-04 19:57:37,2017-10-04 20:07:14,2017-10-05 16:52:52,2017-10-20 20:25:45,2017-11-07
99435,880675dff2150932f1601e1c07eadeeb,47cd45a6ac7b9fb16537df2ccffeb5ac,delivered,2017-02-23 09:05:12,2017-02-23 09:15:11,2017-03-01 10:22:52,2017-03-06 11:08:08,2017-03-22
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27


In [59]:
#jointure pour récupérer le customer_unique_id
ords=ords[['customer_id','order_purchase_timestamp']].merge(customers[['customer_id','customer_unique_id']], how='left')
ords

,customer_id,order_purchase_timestamp,customer_unique_id
0,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff
1,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,7c142cf63193a1473d2e66489a9ae977
2,503740e9ca751ccdda7ba28e9ab8f608,2017-07-09 21:57:05,80bb27c7c16e8f973207a5086ab329e2
3,ed0271e0b7da060a393796590e7b737a,2017-04-11 12:22:08,36edbb3fb164b1f16485364b6fb04c73
4,9bdf08b4b3b52b5526ff42d37d47f222,2017-05-16 13:10:30,932afa1e708222e5821dac9cd5db4cae
...,...,...,...
51309,da2124f134f5dfbce9d06f29bdb6c308,2017-10-04 19:57:37,c716cf2b5b86fb24257cffe9e7969df8
51310,47cd45a6ac7b9fb16537df2ccffeb5ac,2017-02-23 09:05:12,831ce3f1bacbd424fc4e38fbd4d66d29
51311,39bd1228ee8140590ac3aca26f2dfe00,2017-03-09 09:54:05,6359f309b166b0196dbf7ad2ac62bb5a
51312,1aa71eb042121263aafbe80c1b562c9c,2017-08-27 14:46:43,737520a9aad80b3fbbdad19b66b37b30


In [60]:
#classer suivant la date de commande de la plus récente à la plus ancienne & drop duplicates sur customer_unique_id pour 
#récupérer la date de la commande la plus récente
dates=ords.sort_values(by='order_purchase_timestamp', ascending=False)
dates

,customer_id,order_purchase_timestamp,customer_unique_id
18952,3c0f690c8e83b76f461fcc7ca70870ec,2018-01-30 23:59:43,3fb69c9b0644f40f55168ca72a7b01e3
37936,ec201fc4260520724e8ed826a03bfbf9,2018-01-30 23:59:17,71affe163d03c72a07cc3238d6945440
21639,57b7db7fc8a60f117c4ca88ba371d5a0,2018-01-30 23:56:06,dd610cadaf3b8b4d5d685a36b914e4d7
15980,ee346cba5a3efbdd8663133d9ff755e8,2018-01-30 23:42:58,843e80aa83a944d60e7e46829fdc17a1
44651,6d71f3c2700fcf29edc036e602af2088,2018-01-30 23:41:41,3ca2f14e7557aad9076a9056d0c41c5e
...,...,...,...
30480,42da46ef0cdd769b68cea0d88b6bc5ec,2017-02-01 09:08:09,ef057d5e1e98631079064c5b892de94c
22099,628c8188fe28ce092be622bf1361acb7,2017-02-01 08:45:22,4fd565778fe054e28d5aba1088700075
10423,3efd5061c7c8d80b80b3f2b3a5e12f48,2017-02-01 08:42:45,13aec51202205c662b24202760e93a92
17331,9333fb1c5675b69afe49a9cf6b91defe,2017-02-01 08:42:43,13aec51202205c662b24202760e93a92


In [61]:
dates=dates.drop_duplicates(subset=['customer_unique_id'])
dates

,customer_id,order_purchase_timestamp,customer_unique_id
18952,3c0f690c8e83b76f461fcc7ca70870ec,2018-01-30 23:59:43,3fb69c9b0644f40f55168ca72a7b01e3
37936,ec201fc4260520724e8ed826a03bfbf9,2018-01-30 23:59:17,71affe163d03c72a07cc3238d6945440
21639,57b7db7fc8a60f117c4ca88ba371d5a0,2018-01-30 23:56:06,dd610cadaf3b8b4d5d685a36b914e4d7
15980,ee346cba5a3efbdd8663133d9ff755e8,2018-01-30 23:42:58,843e80aa83a944d60e7e46829fdc17a1
44651,6d71f3c2700fcf29edc036e602af2088,2018-01-30 23:41:41,3ca2f14e7557aad9076a9056d0c41c5e
...,...,...,...
48117,e29582b5eb3bee86cc7d5b4f43b63a6a,2017-02-01 09:08:50,4bb740141ad510be830b6c6b7914f25b
30480,42da46ef0cdd769b68cea0d88b6bc5ec,2017-02-01 09:08:09,ef057d5e1e98631079064c5b892de94c
22099,628c8188fe28ce092be622bf1361acb7,2017-02-01 08:45:22,4fd565778fe054e28d5aba1088700075
10423,3efd5061c7c8d80b80b3f2b3a5e12f48,2017-02-01 08:42:45,13aec51202205c662b24202760e93a92


In [62]:
#Calcul du nb de jours depuis le dernier achat
dates.insert(3,'nb_days_last_order','',True)
dates['nb_days_last_order']=dates['order_purchase_timestamp'].apply(lambda x: (date_end-x).days)
dates

<ipython-input-62-9e63b2255ae0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dates['nb_days_last_order']=dates['order_purchase_timestamp'].apply(lambda x: (date_end-x).days)


,customer_id,order_purchase_timestamp,customer_unique_id,nb_days_last_order
18952,3c0f690c8e83b76f461fcc7ca70870ec,2018-01-30 23:59:43,3fb69c9b0644f40f55168ca72a7b01e3,0
37936,ec201fc4260520724e8ed826a03bfbf9,2018-01-30 23:59:17,71affe163d03c72a07cc3238d6945440,0
21639,57b7db7fc8a60f117c4ca88ba371d5a0,2018-01-30 23:56:06,dd610cadaf3b8b4d5d685a36b914e4d7,0
15980,ee346cba5a3efbdd8663133d9ff755e8,2018-01-30 23:42:58,843e80aa83a944d60e7e46829fdc17a1,0
44651,6d71f3c2700fcf29edc036e602af2088,2018-01-30 23:41:41,3ca2f14e7557aad9076a9056d0c41c5e,0
...,...,...,...,...
48117,e29582b5eb3bee86cc7d5b4f43b63a6a,2017-02-01 09:08:50,4bb740141ad510be830b6c6b7914f25b,363
30480,42da46ef0cdd769b68cea0d88b6bc5ec,2017-02-01 09:08:09,ef057d5e1e98631079064c5b892de94c,363
22099,628c8188fe28ce092be622bf1361acb7,2017-02-01 08:45:22,4fd565778fe054e28d5aba1088700075,363
10423,3efd5061c7c8d80b80b3f2b3a5e12f48,2017-02-01 08:42:45,13aec51202205c662b24202760e93a92,363


In [63]:
#jointure avec fichier de travail
work=work.merge(dates[['customer_unique_id','nb_days_last_order']], how='left')
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value,mean_payment_installments,nb_days_last_order
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0,2.0,259.0
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0,8.0,18.0
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0,0.0,NaN
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0,0.0,NaN
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0,0.0,NaN
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0,2.0,88.0


### B.2.b - Fréquence : <a name=B.2.b ></a>
#### B.2.b1 - Nombre de jours moyen entre 2 commandes (delta_moy) <a name=B.2.b1 ></a>
On va récupérer la date de la 1ère et de la dernière commande de chaque client. Puis calculer le delta, date_derniere_commande - date_première_commande. Ensuite divise ce delta par le nombre de commande totales effectuées par chaque client.

In [64]:
#on récupère le customer_unique_id de chaques commandes
ord_cust=orders[['customer_id','order_purchase_timestamp']].merge(customers[['customer_id','customer_unique_id']], how='left')
ord_cust

,customer_id,order_purchase_timestamp,customer_unique_id
0,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff
1,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,af07308b275d755c9edb36a90c618231
2,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,3a653a41f6f9fc3d2a113cf8398680e8
3,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,7c142cf63193a1473d2e66489a9ae977
4,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,72632f0f9dd73dfee390c9b22eb56dd6
...,...,...,...
99436,39bd1228ee8140590ac3aca26f2dfe00,2017-03-09 09:54:05,6359f309b166b0196dbf7ad2ac62bb5a
99437,1fca14ff2861355f6e5f14306ff977a7,2018-02-06 12:58:58,da62f9e57a76d978d02ab5362c509660
99438,1aa71eb042121263aafbe80c1b562c9c,2017-08-27 14:46:43,737520a9aad80b3fbbdad19b66b37b30
99439,b331b74b18dc79bcdf6532d51e1637c1,2018-01-08 21:28:27,5097a5312c8b157bb7be58ae360ef43c


In [65]:
#selection commandes dans la période considérée
ord_cust=ord_cust[(ord_cust['order_purchase_timestamp']>date_start) & (ord_cust['order_purchase_timestamp']<date_end)]
ord_cust

,customer_id,order_purchase_timestamp,customer_unique_id
0,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,7c396fd4830fd04220f754e42b4e5bff
3,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,7c142cf63193a1473d2e66489a9ae977
5,503740e9ca751ccdda7ba28e9ab8f608,2017-07-09 21:57:05,80bb27c7c16e8f973207a5086ab329e2
6,ed0271e0b7da060a393796590e7b737a,2017-04-11 12:22:08,36edbb3fb164b1f16485364b6fb04c73
7,9bdf08b4b3b52b5526ff42d37d47f222,2017-05-16 13:10:30,932afa1e708222e5821dac9cd5db4cae
...,...,...,...
99433,da2124f134f5dfbce9d06f29bdb6c308,2017-10-04 19:57:37,c716cf2b5b86fb24257cffe9e7969df8
99435,47cd45a6ac7b9fb16537df2ccffeb5ac,2017-02-23 09:05:12,831ce3f1bacbd424fc4e38fbd4d66d29
99436,39bd1228ee8140590ac3aca26f2dfe00,2017-03-09 09:54:05,6359f309b166b0196dbf7ad2ac62bb5a
99438,1aa71eb042121263aafbe80c1b562c9c,2017-08-27 14:46:43,737520a9aad80b3fbbdad19b66b37b30


In [66]:
#date dernière commande
last=ord_cust.sort_values(by='order_purchase_timestamp', ascending=False)
last=last[['order_purchase_timestamp','customer_unique_id']]
last=last.drop_duplicates(subset=['customer_unique_id'], keep='first')
last=last.rename(columns={"order_purchase_timestamp": "last_order_date"})
last

,last_order_date,customer_unique_id
36698,2018-01-30 23:59:43,3fb69c9b0644f40f55168ca72a7b01e3
73579,2018-01-30 23:59:17,71affe163d03c72a07cc3238d6945440
41859,2018-01-30 23:56:06,dd610cadaf3b8b4d5d685a36b914e4d7
30983,2018-01-30 23:42:58,843e80aa83a944d60e7e46829fdc17a1
86423,2018-01-30 23:41:41,3ca2f14e7557aad9076a9056d0c41c5e
...,...,...
93190,2017-02-01 09:08:50,4bb740141ad510be830b6c6b7914f25b
59193,2017-02-01 09:08:09,ef057d5e1e98631079064c5b892de94c
42790,2017-02-01 08:45:22,4fd565778fe054e28d5aba1088700075
20245,2017-02-01 08:42:45,13aec51202205c662b24202760e93a92


In [67]:
#date 1ère commande
first=ord_cust.sort_values(by='order_purchase_timestamp', ascending=True)
first=first[['order_purchase_timestamp','customer_unique_id']]
first=first.drop_duplicates(subset=['customer_unique_id'], keep='first')
first=first.rename(columns={"order_purchase_timestamp": "first_order_date"})
first

,first_order_date,customer_unique_id
24321,2017-02-01 00:04:17,0071f46a072a9ae25bbe4438b15efe9c
33546,2017-02-01 08:42:43,13aec51202205c662b24202760e93a92
42790,2017-02-01 08:45:22,4fd565778fe054e28d5aba1088700075
59193,2017-02-01 09:08:09,ef057d5e1e98631079064c5b892de94c
93190,2017-02-01 09:08:50,4bb740141ad510be830b6c6b7914f25b
...,...,...
86423,2018-01-30 23:41:41,3ca2f14e7557aad9076a9056d0c41c5e
30983,2018-01-30 23:42:58,843e80aa83a944d60e7e46829fdc17a1
41859,2018-01-30 23:56:06,dd610cadaf3b8b4d5d685a36b914e4d7
73579,2018-01-30 23:59:17,71affe163d03c72a07cc3238d6945440


In [68]:
#calcul delta
delta=last.merge(first)
delta.insert(3,'delta','',True)
delta['delta']=(delta['last_order_date']-delta['first_order_date']).dt.days
delta.sort_values(by='delta', ascending=False)

,last_order_date,customer_unique_id,first_order_date,delta
2076,2018-01-21 22:51:21,0942eedfe0e865b2a80419a14672286f,2017-02-08 07:47:47,347
2282,2018-01-20 22:09:34,82809cee3c35eb89b4dd4d2d1a22eedf,2017-02-07 12:47:52,347
2025,2018-01-22 10:18:40,8f6ce2295bdbec03cd50e34b4bd7ba0a,2017-02-09 12:48:59,346
4501,2018-01-11 22:14:00,b643c19d91a65d73f94ba8e037668986,2017-02-05 17:43:31,340
4877,2018-01-10 15:18:58,c658495c41a24ef33a852db6840738a1,2017-02-09 10:51:03,335
...,...,...,...,...
16769,2017-11-20 10:59:51,b80150ddbd98f4e62edc8b8f86d4d3a2,2017-11-20 10:59:51,0
16770,2017-11-20 10:59:50,8286a6747b4cfbe5ea998f77f5350fec,2017-11-20 10:59:50,0
16771,2017-11-20 10:59:49,f438882f85fa6497c3e5912724ca16f5,2017-11-20 10:59:49,0
16772,2017-11-20 10:59:42,10c792a88e593bba5b8d390c5a8ad942,2017-11-20 10:59:42,0


In [69]:
#jointure avec le fichier de travail 
work=work.merge(delta[['customer_unique_id','delta']],how='left')
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value,mean_payment_installments,nb_days_last_order,delta
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0,2.0,259.0,0.0
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0,8.0,18.0,0.0
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,NaN
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0,0.0,NaN,NaN
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,NaN
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0,0.0,NaN,NaN
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0,0.0,NaN,NaN
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0,2.0,88.0,0.0


In [70]:
#calcul delta moyen
work['delta']=np.round(work['delta']/(work['nb_orders']-1))
work=work.rename(columns={"delta": "delta_moy"})
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value,mean_payment_installments,nb_days_last_order,delta_moy
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0,2.0,259.0,NaN
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0,8.0,18.0,NaN
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,NaN
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0,0.0,NaN,NaN
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,NaN
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0,0.0,NaN,NaN
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0,0.0,NaN,NaN
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0,2.0,88.0,NaN


Le problème avec cette variable c'est qu'elle est valable uniquement si les clients ont effectués au moins 2 commandes, en effet, on ne peux pas calculer de delta moyen entre 2 commandes si le nombre de commandes est inférieur à 2. Aussi vouloir valoriser les données manquantes pour cette variable et pour les individus ayant effectués 1 commande car ça n'a pas de sens. Je vais donc retirer cette variable du fichier de travail. 

In [71]:
work=work.drop(columns=['delta_moy'])
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value,mean_payment_installments,nb_days_last_order
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0,2.0,259.0
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0,8.0,18.0
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0,0.0,NaN
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0,0.0,NaN
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0,0.0,NaN
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0,2.0,88.0


#### B.2.b2 - Catégories des produits les plus commandés (most_freq_cat_name) <a name=B.2.b2 ></a>

In [72]:
#récupération des classes des produits commandés
order_items_cat_name=order_items.merge(products, how='left')[['order_id','order_item_id','product_id','product_category_name']]
order_items_cat_name

,order_id,order_item_id,product_id,product_category_name
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,pet_shop
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,moveis_decoracao
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,perfumaria
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,ferramentas_jardim
...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,utilidades_domesticas
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,informatica_acessorios
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,esporte_lazer
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,informatica_acessorios


In [73]:
#récuperer le customer_id et la date d'achat pour chaques produits commandés
orders_cat_name=order_items_cat_name[['order_id','product_category_name']].merge(orders[['order_id','customer_id','order_purchase_timestamp']], how='left')
orders_cat_name

,order_id,product_category_name,customer_id,order_purchase_timestamp
0,00010242fe8c5a6d1ba2dd792cb16214,cool_stuff,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02
1,00018f77f2f0320c557190d7a144bdd3,pet_shop,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06
2,000229ec398224ef6ca0657da4fc703e,moveis_decoracao,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31
3,00024acbcdf0a6daa1e931b038114c75,perfumaria,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35
4,00042b26cf59d7ce69dfabb4e55b4fd9,ferramentas_jardim,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51
...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,utilidades_domesticas,b51593916b4b8e0d6f66f2ae24f2673d,2018-04-23 13:57:06
112646,fffcd46ef2263f404302a634eb57f7eb,informatica_acessorios,84c5d4fbaf120aae381fad077416eaa0,2018-07-14 10:26:46
112647,fffce4705a9662cd70adb13d4a31832d,esporte_lazer,29309aa813182aaddc9b259e31b870e6,2017-10-23 17:07:56
112648,fffe18544ffabc95dfada21779c9644f,informatica_acessorios,b5e6afd5a41800fdf401e0272ca74655,2017-08-14 23:02:59


In [74]:
#filtrer les commandes qui correspondent à la période sélectionnée
orders_cat_name=orders_cat_name[(orders_cat_name['order_purchase_timestamp']>date_start) & (orders_cat_name['order_purchase_timestamp']<date_end)]
orders_cat_name

,order_id,product_category_name,customer_id,order_purchase_timestamp
0,00010242fe8c5a6d1ba2dd792cb16214,cool_stuff,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02
1,00018f77f2f0320c557190d7a144bdd3,pet_shop,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06
2,000229ec398224ef6ca0657da4fc703e,moveis_decoracao,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31
4,00042b26cf59d7ce69dfabb4e55b4fd9,ferramentas_jardim,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51
5,00048cc3ae777c65dbb7d2a0634bc1ea,utilidades_domesticas,816cbea969fe5b689b39cfc97a506742,2017-05-15 21:42:34
...,...,...,...,...
112641,fffb9224b6fc7c43ebb0904318b10b5f,relogios_presentes,4d3abb73ceb86353aeadbe698aa9d5cb,2017-10-27 16:51:00
112642,fffb9224b6fc7c43ebb0904318b10b5f,relogios_presentes,4d3abb73ceb86353aeadbe698aa9d5cb,2017-10-27 16:51:00
112643,fffb9224b6fc7c43ebb0904318b10b5f,relogios_presentes,4d3abb73ceb86353aeadbe698aa9d5cb,2017-10-27 16:51:00
112647,fffce4705a9662cd70adb13d4a31832d,esporte_lazer,29309aa813182aaddc9b259e31b870e6,2017-10-23 17:07:56


In [75]:
#on récupère les customer_unique_id
orders_cat_name=orders_cat_name.drop(columns=['order_purchase_timestamp'])
orders_cat_name=orders_cat_name.merge(customers[['customer_id','customer_unique_id']], how='left')
orders_cat_name

,order_id,product_category_name,customer_id,customer_unique_id
0,00010242fe8c5a6d1ba2dd792cb16214,cool_stuff,3ce436f183e68e07877b285a838db11a,871766c5855e863f6eccc05f988b23cb
1,00018f77f2f0320c557190d7a144bdd3,pet_shop,f6dd3ec061db4e3987629fe6b26e5cce,eb28e67c4c0b83846050ddfb8a35d051
2,000229ec398224ef6ca0657da4fc703e,moveis_decoracao,6489ae5e4333f3693df5ad4372dab6d3,3818d81c6709e39d06b2738a8d3a2474
3,00042b26cf59d7ce69dfabb4e55b4fd9,ferramentas_jardim,58dbd0b2d70206bf40e62cd34e84d795,64b576fb70d441e8f1b2d7d446e483c5
4,00048cc3ae777c65dbb7d2a0634bc1ea,utilidades_domesticas,816cbea969fe5b689b39cfc97a506742,85c835d128beae5b4ce8602c491bf385
...,...,...,...,...
57828,fffb9224b6fc7c43ebb0904318b10b5f,relogios_presentes,4d3abb73ceb86353aeadbe698aa9d5cb,f736308cd9952b33b90b9fe94da9c8f5
57829,fffb9224b6fc7c43ebb0904318b10b5f,relogios_presentes,4d3abb73ceb86353aeadbe698aa9d5cb,f736308cd9952b33b90b9fe94da9c8f5
57830,fffb9224b6fc7c43ebb0904318b10b5f,relogios_presentes,4d3abb73ceb86353aeadbe698aa9d5cb,f736308cd9952b33b90b9fe94da9c8f5
57831,fffce4705a9662cd70adb13d4a31832d,esporte_lazer,29309aa813182aaddc9b259e31b870e6,cd79b407828f02fdbba457111c38e4c4


In [76]:
#catégories les plus commandées
most_freq_cat_name=pd.DataFrame(orders_cat_name.groupby('customer_unique_id')['product_category_name'].value_counts())
most_freq_cat_name = most_freq_cat_name.rename(columns={"product_category_name": "most_freq_cat_name"})
most_freq_cat_name=most_freq_cat_name.reset_index()
most_freq_cat_name

,customer_unique_id,product_category_name,most_freq_cat_name
0,0000f46a3911fa3c0805444483337064,papelaria,1
1,0000f6ccb0745a6a4b88665a16c9f078,telefonia,1
2,0004aac84e0df4da2b147fca70cf8255,telefonia,1
3,0005e1862207bf6ccc02e4228effd9a0,fashion_bolsas_e_acessorios,1
4,0006fdc98a402fceb4eb0ee528f6a8d4,cama_mesa_banho,1
...,...,...,...
49352,fffb09418989a0dbff854a28163e47c6,relogios_presentes,1
49353,fffcf5a5ff07b0908bd4e2dbc735a684,beleza_saude,2
49354,fffea47cd6d3cc0a88bd621562a9d061,bebes,1
49355,ffff371b4d645b6ecea244b27531430a,automotivo,1


In [77]:
#on classe suivant le nombre d'apparition de la catégorie
most_freq_cat_name=most_freq_cat_name.sort_values(by='most_freq_cat_name', ascending=False)
most_freq_cat_name

,customer_unique_id,product_category_name,most_freq_cat_name
13363,4546caea018ad8c692964e3382debd19,beleza_saude,21
20295,698e1cf81d01a3d389d96145f7fa6df8,automotivo,20
2915,0f5ac8d5c31de21d2f25e24be15bbffb,moveis_decoracao,18
9607,31e412b9fb766b6794724ed17a41dfa6,ferramentas_jardim,14
42023,d97b3cfb22b0d6b25ac9ed4e9c2d481b,cama_mesa_banho,13
...,...,...,...
17333,59b2394f4f369bc6ebdd38a5de11c927,informatica_acessorios,1
17334,59b265b9cfda8d25bbc84f722b90b139,informatica_acessorios,1
17335,59b46e4ceb1bca705bbae5644d5918db,beleza_saude,1
17336,59b5158dcd4ab8fce908aba30ea36cc9,cool_stuff,1


In [78]:
#On supprime les doublons de customer_unique_id
most_freq_cat_name=most_freq_cat_name.drop_duplicates(subset=['customer_unique_id'], keep='first')
most_freq_cat_name

,customer_unique_id,product_category_name,most_freq_cat_name
13363,4546caea018ad8c692964e3382debd19,beleza_saude,21
20295,698e1cf81d01a3d389d96145f7fa6df8,automotivo,20
2915,0f5ac8d5c31de21d2f25e24be15bbffb,moveis_decoracao,18
9607,31e412b9fb766b6794724ed17a41dfa6,ferramentas_jardim,14
42023,d97b3cfb22b0d6b25ac9ed4e9c2d481b,cama_mesa_banho,13
...,...,...,...
17333,59b2394f4f369bc6ebdd38a5de11c927,informatica_acessorios,1
17334,59b265b9cfda8d25bbc84f722b90b139,informatica_acessorios,1
17335,59b46e4ceb1bca705bbae5644d5918db,beleza_saude,1
17336,59b5158dcd4ab8fce908aba30ea36cc9,cool_stuff,1


In [79]:
#On récupère le nom anglais des catégories :
most_freq_cat_name=most_freq_cat_name.merge(cat_name, how='left')
most_freq_cat_name

,customer_unique_id,product_category_name,most_freq_cat_name,product_category_name_english
0,4546caea018ad8c692964e3382debd19,beleza_saude,21,health_beauty
1,698e1cf81d01a3d389d96145f7fa6df8,automotivo,20,auto
2,0f5ac8d5c31de21d2f25e24be15bbffb,moveis_decoracao,18,furniture_decor
3,31e412b9fb766b6794724ed17a41dfa6,ferramentas_jardim,14,garden_tools
4,d97b3cfb22b0d6b25ac9ed4e9c2d481b,cama_mesa_banho,13,bed_bath_table
...,...,...,...,...
48314,59b2394f4f369bc6ebdd38a5de11c927,informatica_acessorios,1,computers_accessories
48315,59b265b9cfda8d25bbc84f722b90b139,informatica_acessorios,1,computers_accessories
48316,59b46e4ceb1bca705bbae5644d5918db,beleza_saude,1,health_beauty
48317,59b5158dcd4ab8fce908aba30ea36cc9,cool_stuff,1,cool_stuff


In [80]:
#jointure avec le fichier de travail
work=work.merge(most_freq_cat_name[['product_category_name_english','customer_unique_id']], how='left')
work['product_category_name_english']=work['product_category_name_english'].fillna('missing')
work=work.rename(columns={"product_category_name_english": "most_freq_cat_name"})
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value,mean_payment_installments,nb_days_last_order,most_freq_cat_name
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0,2.0,259.0,office_furniture
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0,8.0,18.0,housewares
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing
...,...,...,...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0,2.0,88.0,watches_gifts


### B.2.c - Montant : <a name=B.2.c ></a>
#### B.2.c1 - Montant total des commandes  (payment_value) <a name=B.2.c1 ></a>

In [81]:
#jointure entre orders & order_payments pour avoir le montant du paiment & le customer_id
orders_values=order_payments[['order_id','payment_value']].merge(orders[['order_id','customer_id','order_purchase_timestamp']], how='left')
orders_values

,order_id,payment_value,customer_id,order_purchase_timestamp
0,b81ef226f3fe1789b1e8b2acac839d17,99.33,0a8556ac6be836b46b3e89920d59291c,2018-04-25 22:01:49
1,a9810da82917af2d9aefd1278f1dcfa0,24.39,f2c7fc58a9de810828715166c672f10a,2018-06-26 11:01:38
2,25e8ea4e93396b6fa0d3dd708e76c1bd,65.71,25b14b69de0b6e184ae6fe2755e478f9,2017-12-12 11:19:55
3,ba78997921bbcdc1373bb41e913ab953,107.78,7a5d8efaaa1081f800628c30d2b0728f,2017-12-06 12:04:06
4,42fdf880ba16b47b59251dd489d4441a,128.45,15fd6fb8f8312dbb4674e4518d6fa3b3,2018-05-21 13:59:17
...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,363.31,5d576cb2dfa3bc05612c392a1ee9c654,2018-03-08 16:57:23
103882,7b905861d7c825891d6347454ea7863f,96.80,2079230c765a88530822a34a4cec2aa0,2017-08-18 09:45:35
103883,32609bbb3dd69b3c066a6860554a77bf,47.77,e4abb5057ec8cfda9759c0dc415a8188,2017-11-18 17:27:14
103884,b8b61059626efa996a60be9bb9320e10,369.54,5d719b0d300663188169c6560e243f27,2018-08-07 23:26:13


In [82]:
#filtre pour récupérer les commandes sur la période considérée
orders_values=orders_values[(orders_values['order_purchase_timestamp']>date_start) & (orders_values['order_purchase_timestamp']<date_end)]
orders_values

,order_id,payment_value,customer_id,order_purchase_timestamp
2,25e8ea4e93396b6fa0d3dd708e76c1bd,65.71,25b14b69de0b6e184ae6fe2755e478f9,2017-12-12 11:19:55
3,ba78997921bbcdc1373bb41e913ab953,107.78,7a5d8efaaa1081f800628c30d2b0728f,2017-12-06 12:04:06
6,771ee386b001f06208a7419e4fc1bbd7,81.16,206c0fcc717333d169ff95933fb47341,2017-06-23 13:11:27
7,3d7239c394a212faae122962df514ac7,51.84,4d7a9b8bba459dce09d1b8fa637a0cba,2017-06-05 10:45:54
9,0573b5e23cbd798006520e1d5b4c6714,51.95,c4e17eff78176dfe3401f03db0346f85,2017-07-04 20:32:10
...,...,...,...,...
103876,0872d0faafe0cb56e6f2c594cca1522c,127.92,dd2f0d64271438f6ad3a75d2a930a4b5,2017-09-26 12:10:26
103878,2e3d96522c8c378d4135c12a65c977aa,47.62,cd0a090974c3b64acf613c18f9fcfe83,2017-11-23 17:11:23
103882,7b905861d7c825891d6347454ea7863f,96.80,2079230c765a88530822a34a4cec2aa0,2017-08-18 09:45:35
103883,32609bbb3dd69b3c066a6860554a77bf,47.77,e4abb5057ec8cfda9759c0dc415a8188,2017-11-18 17:27:14


In [83]:
#jointure entre orders_values & customers pour avoir le montant des commandes & le customer_unique_id
orders_values=orders_values[['customer_id','payment_value']].merge(customers[['customer_id','customer_unique_id']], how='left')
orders_values

,customer_id,payment_value,customer_unique_id
0,25b14b69de0b6e184ae6fe2755e478f9,65.71,6f70c0b2f7552832ba46eb57b1c5651e
1,7a5d8efaaa1081f800628c30d2b0728f,107.78,87695ed086ebd36f20404c82d20fca87
2,206c0fcc717333d169ff95933fb47341,81.16,9a3526eee723accd014ec683afe5bf20
3,4d7a9b8bba459dce09d1b8fa637a0cba,51.84,9db67268a3fee1d4b13faaeb90af07c0
4,c4e17eff78176dfe3401f03db0346f85,51.95,511c345b0383af6ef25406c564e4d6b6
...,...,...,...
53970,dd2f0d64271438f6ad3a75d2a930a4b5,127.92,4a57b2a3676de278e26a50b7e60214e6
53971,cd0a090974c3b64acf613c18f9fcfe83,47.62,1c157ec47b1201b33c6ea9cc61c966fb
53972,2079230c765a88530822a34a4cec2aa0,96.80,53b30ca78efb2b7efcd3f9e461587eb2
53973,e4abb5057ec8cfda9759c0dc415a8188,47.77,d3c7da954a324253814096bcaf240e4e


In [84]:
#calcul de la somme totale des commandes réalisé par chaque client
pay_value=pd.DataFrame(orders_values.groupby('customer_unique_id')['payment_value'].sum()).reset_index()
pay_value

,customer_unique_id,payment_value
0,0000f46a3911fa3c0805444483337064,86.22
1,0000f6ccb0745a6a4b88665a16c9f078,43.62
2,0004aac84e0df4da2b147fca70cf8255,196.89
3,0005e1862207bf6ccc02e4228effd9a0,150.12
4,0006fdc98a402fceb4eb0ee528f6a8d4,29.00
...,...,...
49744,fffbf87b7a1a6fa8b03f081c5f51a201,167.32
49745,fffcf5a5ff07b0908bd4e2dbc735a684,2067.42
49746,fffea47cd6d3cc0a88bd621562a9d061,84.58
49747,ffff371b4d645b6ecea244b27531430a,112.46


In [85]:
#jointure avec le fichier de travail
work=work.merge(pay_value, how='left')
work['payment_value']=work['payment_value'].fillna(0.0)
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value,mean_payment_installments,nb_days_last_order,most_freq_cat_name,payment_value
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0,2.0,259.0,office_furniture,146.87
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0,8.0,18.0,housewares,335.48
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0,2.0,88.0,watches_gifts,711.07


#### B.2.c2 - Panier : <a name=B.2.c2 ></a>
##### B.2.c2.a - Montant du panier moyen (average_basket) <a name=B.2.c2.a ></a>
Tout simplement le montant de toute le commandes réalisés par le client divisé par le nombre de commandes effectuées

In [86]:
work.insert(12,'average_basket','',True)
work['average_basket']=work['payment_value']/work['nb_orders']
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value,mean_payment_installments,nb_days_last_order,most_freq_cat_name,payment_value,average_basket
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0,2.0,259.0,office_furniture,146.87,146.87
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0,8.0,18.0,housewares,335.48,335.48
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0,2.0,88.0,watches_gifts,711.07,711.07


##### B.2.c2.b - Montant du panier minimum (min_basket) <a name=B.2.c2.b ></a>

In [87]:
#Calcul des montant de chaques commandes comme il y a plusieurs order_id
pay_max=pd.DataFrame(order_payments.groupby('order_id')['payment_value'].sum())
pay_max=pay_max.reset_index()
pay_max

,order_id,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,72.19
1,00018f77f2f0320c557190d7a144bdd3,259.83
2,000229ec398224ef6ca0657da4fc703e,216.87
3,00024acbcdf0a6daa1e931b038114c75,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04
...,...,...
99435,fffc94f6ce00a00581880bf54a75a037,343.40
99436,fffcd46ef2263f404302a634eb57f7eb,386.53
99437,fffce4705a9662cd70adb13d4a31832d,116.85
99438,fffe18544ffabc95dfada21779c9644f,64.71


In [88]:
#on récupère le customer_id et la date de commande
pay_max=pay_max.merge(orders[['order_id','customer_id','order_purchase_timestamp']], how='left')
pay_max

,order_id,payment_value,customer_id,order_purchase_timestamp
0,00010242fe8c5a6d1ba2dd792cb16214,72.19,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02
1,00018f77f2f0320c557190d7a144bdd3,259.83,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06
2,000229ec398224ef6ca0657da4fc703e,216.87,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31
3,00024acbcdf0a6daa1e931b038114c75,25.78,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51
...,...,...,...,...
99435,fffc94f6ce00a00581880bf54a75a037,343.40,b51593916b4b8e0d6f66f2ae24f2673d,2018-04-23 13:57:06
99436,fffcd46ef2263f404302a634eb57f7eb,386.53,84c5d4fbaf120aae381fad077416eaa0,2018-07-14 10:26:46
99437,fffce4705a9662cd70adb13d4a31832d,116.85,29309aa813182aaddc9b259e31b870e6,2017-10-23 17:07:56
99438,fffe18544ffabc95dfada21779c9644f,64.71,b5e6afd5a41800fdf401e0272ca74655,2017-08-14 23:02:59


In [89]:
#on filtre les commandes suivant la période considérée
pay_max=pay_max[(pay_max['order_purchase_timestamp']>date_start) & (pay_max['order_purchase_timestamp']<date_end)]
pay_max

,order_id,payment_value,customer_id,order_purchase_timestamp
0,00010242fe8c5a6d1ba2dd792cb16214,72.19,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02
1,00018f77f2f0320c557190d7a144bdd3,259.83,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06
2,000229ec398224ef6ca0657da4fc703e,216.87,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51
5,00048cc3ae777c65dbb7d2a0634bc1ea,34.59,816cbea969fe5b689b39cfc97a506742,2017-05-15 21:42:34
...,...,...,...,...
99430,fffa82886406ccf10c7b4e35c4ff2788,273.92,a5201e1a6d71a8d21e869151bd5b4085,2017-12-18 16:33:07
99431,fffb0b1a50e65c449020434fa835e078,15.86,46b385c6d972b73cbcf797a8b16c072d,2017-04-12 16:35:43
99433,fffb9224b6fc7c43ebb0904318b10b5f,356.76,4d3abb73ceb86353aeadbe698aa9d5cb,2017-10-27 16:51:00
99437,fffce4705a9662cd70adb13d4a31832d,116.85,29309aa813182aaddc9b259e31b870e6,2017-10-23 17:07:56


In [90]:
#on récupère le customer_unique_id
pay_max=pay_max[['payment_value','customer_id']].merge(customers[['customer_id','customer_unique_id']],how='left')
pay_max

,payment_value,customer_id,customer_unique_id
0,72.19,3ce436f183e68e07877b285a838db11a,871766c5855e863f6eccc05f988b23cb
1,259.83,f6dd3ec061db4e3987629fe6b26e5cce,eb28e67c4c0b83846050ddfb8a35d051
2,216.87,6489ae5e4333f3693df5ad4372dab6d3,3818d81c6709e39d06b2738a8d3a2474
3,218.04,58dbd0b2d70206bf40e62cd34e84d795,64b576fb70d441e8f1b2d7d446e483c5
4,34.59,816cbea969fe5b689b39cfc97a506742,85c835d128beae5b4ce8602c491bf385
...,...,...,...
51309,273.92,a5201e1a6d71a8d21e869151bd5b4085,2a3ab9bf9639491997586882c502540a
51310,15.86,46b385c6d972b73cbcf797a8b16c072d,b2d2b4204f7fc4b4aeca38e3cd680a7e
51311,356.76,4d3abb73ceb86353aeadbe698aa9d5cb,f736308cd9952b33b90b9fe94da9c8f5
51312,116.85,29309aa813182aaddc9b259e31b870e6,cd79b407828f02fdbba457111c38e4c4


In [91]:
#panier minimum, on ordonne les commandes de la plus petite a la plus grande et on supprime les doublons de customer_unique_id
min_basket=pay_max.sort_values(by='payment_value', ascending=True)
min_basket=min_basket.drop_duplicates(subset=['customer_unique_id'], keep='first')
min_basket=min_basket.rename(columns={"payment_value": "min_basket"})
min_basket

,min_basket,customer_id,customer_unique_id
46650,10.07,184e8e8e48937145eb96c721ef1f0747,bd06ce0e06ad77a7f681f1a4960a3cc6
27926,11.63,c466c7e0ab222e3ef6c8046e96128a8d,6f5b9d1cdccc4d28f0483a612edecacf
7924,11.63,55cd7bfe95dcd698acf176278e14888e,2878e5b88167faab17d4fb83a986d38b
40024,12.22,fe4176df564772c058286208c0daa2a8,a78c7a68877d33bacf81b01bc4a4d2f2
13743,12.28,04ba9496f04b0eaa070def5b5ab662ac,809ca96e9696b9be5f69cd7ae803049d
...,...,...,...
5069,4764.34,24bbf5fd2f2e1b359ee7de94defc4a15,eebb5dda148d3893cdaf5b5ca3040ccb
8887,6081.54,05455dfa7cd02f13d132aa7a6a9729c6,4007669dec559734d6f53e029e360987
49141,6726.66,3fd6777bbce08a352fddd04e4a7cc8f6,ff4159b92c40ebe40454e3e6a7c35ed6
1595,6929.31,c6e2731c5b391845f6800c97401a43a9,dc4802a71eae9be1dd28f5d788ceb526


In [92]:
#jointure avec le fichier de travail
work=work.merge(min_basket[['customer_unique_id','min_basket']],how='left')
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value,mean_payment_installments,nb_days_last_order,most_freq_cat_name,payment_value,average_basket,min_basket
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0,2.0,259.0,office_furniture,146.87,146.87,146.87
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0,8.0,18.0,housewares,335.48,335.48,335.48
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0,2.0,88.0,watches_gifts,711.07,711.07,711.07


##### B.2.c2.c - Montant du panier maximum (max_basket) <a name=B.2.c2.c ></a>

In [93]:
#on ordonne par ordre décroissant le dataframe obtenu précèdement
max_basket=pay_max.sort_values(by='payment_value', ascending=False)
max_basket=max_basket.drop_duplicates(subset=['customer_unique_id'], keep='first')
max_basket=max_basket.rename(columns={"payment_value": "max_basket"})
max_basket

,max_basket,customer_id,customer_unique_id
751,13664.08,1617b1357756262bfa56ab541c47bc16,0a0a92112bd4c708ca5fde585afaa872
1595,6929.31,c6e2731c5b391845f6800c97401a43a9,dc4802a71eae9be1dd28f5d788ceb526
49141,6726.66,3fd6777bbce08a352fddd04e4a7cc8f6,ff4159b92c40ebe40454e3e6a7c35ed6
8887,6081.54,05455dfa7cd02f13d132aa7a6a9729c6,4007669dec559734d6f53e029e360987
33819,4950.34,df55c14d1476a9a3467f131269c2477f,da122df9eeddfedc1dc1f5349a1a690c
...,...,...,...
24890,12.89,58456fe345a47eddb73e11a8c91a1b69,defbae0ddd4aeca563944124fbcc4bae
13743,12.28,04ba9496f04b0eaa070def5b5ab662ac,809ca96e9696b9be5f69cd7ae803049d
27926,11.63,c466c7e0ab222e3ef6c8046e96128a8d,6f5b9d1cdccc4d28f0483a612edecacf
7924,11.63,55cd7bfe95dcd698acf176278e14888e,2878e5b88167faab17d4fb83a986d38b


In [94]:
#jointure avec le fichier de travail
work=work.merge(max_basket[['customer_unique_id','max_basket']],how='left')
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value,mean_payment_installments,nb_days_last_order,most_freq_cat_name,payment_value,average_basket,min_basket,max_basket
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0,2.0,259.0,office_furniture,146.87,146.87,146.87,146.87
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0,8.0,18.0,housewares,335.48,335.48,335.48,335.48
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0,2.0,88.0,watches_gifts,711.07,711.07,711.07,711.07


#### B.2.c3 - Frais de port : <a name=B.2.c3 ></a>
##### B.2.c3.a - Total des frais de port (sum_fdp) <a name=B.2.c3.a ></a>

In [95]:
# frais de port pour chaques commandes
fdp=pd.DataFrame(order_items.groupby('order_id')['freight_value'].sum())
fdp=fdp.reset_index()
fdp

,order_id,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,13.29
1,00018f77f2f0320c557190d7a144bdd3,19.93
2,000229ec398224ef6ca0657da4fc703e,17.87
3,00024acbcdf0a6daa1e931b038114c75,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,18.14
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,43.41
98662,fffcd46ef2263f404302a634eb57f7eb,36.53
98663,fffce4705a9662cd70adb13d4a31832d,16.95
98664,fffe18544ffabc95dfada21779c9644f,8.72


In [96]:
# récupérer les dates de commandes & customer_id
fdp=fdp.merge(orders[['order_id','customer_id','order_purchase_timestamp']], how='left')
fdp

,order_id,freight_value,customer_id,order_purchase_timestamp
0,00010242fe8c5a6d1ba2dd792cb16214,13.29,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02
1,00018f77f2f0320c557190d7a144bdd3,19.93,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06
2,000229ec398224ef6ca0657da4fc703e,17.87,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31
3,00024acbcdf0a6daa1e931b038114c75,12.79,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35
4,00042b26cf59d7ce69dfabb4e55b4fd9,18.14,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51
...,...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,43.41,b51593916b4b8e0d6f66f2ae24f2673d,2018-04-23 13:57:06
98662,fffcd46ef2263f404302a634eb57f7eb,36.53,84c5d4fbaf120aae381fad077416eaa0,2018-07-14 10:26:46
98663,fffce4705a9662cd70adb13d4a31832d,16.95,29309aa813182aaddc9b259e31b870e6,2017-10-23 17:07:56
98664,fffe18544ffabc95dfada21779c9644f,8.72,b5e6afd5a41800fdf401e0272ca74655,2017-08-14 23:02:59


In [97]:
#filtrer les commandes en fonction de la date
fdp=fdp[(fdp['order_purchase_timestamp']>date_start) & (fdp['order_purchase_timestamp']<date_end)]
fdp

,order_id,freight_value,customer_id,order_purchase_timestamp
0,00010242fe8c5a6d1ba2dd792cb16214,13.29,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02
1,00018f77f2f0320c557190d7a144bdd3,19.93,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06
2,000229ec398224ef6ca0657da4fc703e,17.87,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31
4,00042b26cf59d7ce69dfabb4e55b4fd9,18.14,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51
5,00048cc3ae777c65dbb7d2a0634bc1ea,12.69,816cbea969fe5b689b39cfc97a506742,2017-05-15 21:42:34
...,...,...,...,...
98656,fffa82886406ccf10c7b4e35c4ff2788,44.02,a5201e1a6d71a8d21e869151bd5b4085,2017-12-18 16:33:07
98657,fffb0b1a50e65c449020434fa835e078,10.96,46b385c6d972b73cbcf797a8b16c072d,2017-04-12 16:35:43
98659,fffb9224b6fc7c43ebb0904318b10b5f,136.76,4d3abb73ceb86353aeadbe698aa9d5cb,2017-10-27 16:51:00
98663,fffce4705a9662cd70adb13d4a31832d,16.95,29309aa813182aaddc9b259e31b870e6,2017-10-23 17:07:56


In [98]:
#jointure avec les customers pour récupérer le customer_unique_id
fdp=fdp[['freight_value','customer_id']].merge(customers[['customer_id','customer_unique_id']], how='left')
fdp

,freight_value,customer_id,customer_unique_id
0,13.29,3ce436f183e68e07877b285a838db11a,871766c5855e863f6eccc05f988b23cb
1,19.93,f6dd3ec061db4e3987629fe6b26e5cce,eb28e67c4c0b83846050ddfb8a35d051
2,17.87,6489ae5e4333f3693df5ad4372dab6d3,3818d81c6709e39d06b2738a8d3a2474
3,18.14,58dbd0b2d70206bf40e62cd34e84d795,64b576fb70d441e8f1b2d7d446e483c5
4,12.69,816cbea969fe5b689b39cfc97a506742,85c835d128beae5b4ce8602c491bf385
...,...,...,...
50750,44.02,a5201e1a6d71a8d21e869151bd5b4085,2a3ab9bf9639491997586882c502540a
50751,10.96,46b385c6d972b73cbcf797a8b16c072d,b2d2b4204f7fc4b4aeca38e3cd680a7e
50752,136.76,4d3abb73ceb86353aeadbe698aa9d5cb,f736308cd9952b33b90b9fe94da9c8f5
50753,16.95,29309aa813182aaddc9b259e31b870e6,cd79b407828f02fdbba457111c38e4c4


In [99]:
#calcul de la somme de tout les fdp pour chaques clients
sum_fdp=pd.DataFrame(fdp.groupby('customer_unique_id')['freight_value'].sum())
sum_fdp=sum_fdp.reset_index()
sum_fdp=sum_fdp.rename(columns={"freight_value": "sum_fdp"})
sum_fdp

,customer_unique_id,sum_fdp
0,0000f46a3911fa3c0805444483337064,17.22
1,0000f6ccb0745a6a4b88665a16c9f078,17.63
2,0004aac84e0df4da2b147fca70cf8255,16.89
3,0005e1862207bf6ccc02e4228effd9a0,15.12
4,0006fdc98a402fceb4eb0ee528f6a8d4,15.10
...,...,...
49223,fffbf87b7a1a6fa8b03f081c5f51a201,18.32
49224,fffcf5a5ff07b0908bd4e2dbc735a684,497.42
49225,fffea47cd6d3cc0a88bd621562a9d061,19.69
49226,ffff371b4d645b6ecea244b27531430a,22.56


In [100]:
#jointure avec le fichier de travail
work=work.merge(sum_fdp,how='left')
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value,mean_payment_installments,nb_days_last_order,most_freq_cat_name,payment_value,average_basket,min_basket,max_basket,sum_fdp
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0,2.0,259.0,office_furniture,146.87,146.87,146.87,146.87,21.88
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0,8.0,18.0,housewares,335.48,335.48,335.48,335.48,46.48
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0,2.0,88.0,watches_gifts,711.07,711.07,711.07,711.07,22.07


##### B.2.c3.b - Frais de port moyens (mean_fdp)<a name=B.2.c3.b ></a>

In [101]:
#calcuk fdp moyen (sum_fdp/nb_orders)
work.insert(16,'mean_fdp','',True)
work['mean_fdp']=work['sum_fdp']/work['nb_orders']
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value,mean_payment_installments,nb_days_last_order,most_freq_cat_name,payment_value,average_basket,min_basket,max_basket,sum_fdp,mean_fdp
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0,2.0,259.0,office_furniture,146.87,146.87,146.87,146.87,21.88,21.88
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0,8.0,18.0,housewares,335.48,335.48,335.48,335.48,46.48,46.48
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0,2.0,88.0,watches_gifts,711.07,711.07,711.07,711.07,22.07,22.07


##### B.2.c3.c - Frais de port minimum (fdp_min) <a name=B.2.c3.c ></a>

In [102]:
#fdp minimums
fdp_min=fdp.sort_values(by='freight_value', ascending=True)
fdp_min=fdp_min.drop_duplicates(subset=['customer_unique_id'], keep='first')
fdp_min=fdp_min.rename(columns={"freight_value": "fdp_min"})
fdp_min

,fdp_min,customer_id,customer_unique_id
42924,0.00,65a23aa6cfab2172891731a229c0958a,3041bf14421b7d218b3daa4a909199c8
26896,0.00,f03c9c80de7a0a901cd92d8507717007,72b74e3ab8b2f7da203dbb50344879bb
9300,0.00,6a7ca7365d53fad7a6b7dd45406112de,c9b9e60e5338143a868611fd96e30b8f
33409,0.00,baa47b562bdd4118b6b923aa24b14119,040771bb467e731e2f7946b575c319be
29232,0.00,03a81070a9a6e3bc2c4209e564764204,0e50810aa0fdb3784e55bda1bb0a5874
...,...,...,...
26443,436.24,2d54c241a27d5c40c9adf389868e0995,3895f60f6e6a89e5cfb7b72ffdcdf7e0
37683,479.28,0c792d32a3251b4f69dae8646dfbedbc,6d394722d5fc5e721aee6875a218d8db
18165,497.08,42c0b09d094220a110c0e0e941f76f06,527f7f3237fb1397c459701bc765b6f0
22663,497.42,74be082247cd677a147d83ee670e9d53,fffcf5a5ff07b0908bd4e2dbc735a684


In [103]:
work=work.merge(fdp_min[['fdp_min','customer_unique_id']], how='left')
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value,mean_payment_installments,nb_days_last_order,most_freq_cat_name,payment_value,average_basket,min_basket,max_basket,sum_fdp,mean_fdp,fdp_min
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0,2.0,259.0,office_furniture,146.87,146.87,146.87,146.87,21.88,21.88,21.88
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0,8.0,18.0,housewares,335.48,335.48,335.48,335.48,46.48,46.48,46.48
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN,NaN
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN,NaN
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN,NaN
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN,NaN
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN,NaN
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0,2.0,88.0,watches_gifts,711.07,711.07,711.07,711.07,22.07,22.07,22.07


##### B.2.c3.d - Frais de port maximum (fdp_max) <a name=B.2.c3.d ></a>

In [104]:
#fdp max
fdp_max=fdp.sort_values(by='freight_value', ascending=False)
fdp_max=fdp_max.drop_duplicates(subset=['customer_unique_id'], keep='first')
fdp_max=fdp_max.rename(columns={"freight_value": "fdp_max"})
fdp_max

,fdp_max,customer_id,customer_unique_id
7132,1002.29,5e312b0d9bf5c91599eb42120e633b19,066ee6b9c6fc284260ff9a1274a82ca7
22663,497.42,74be082247cd677a147d83ee670e9d53,fffcf5a5ff07b0908bd4e2dbc735a684
18165,497.08,42c0b09d094220a110c0e0e941f76f06,527f7f3237fb1397c459701bc765b6f0
37683,479.28,0c792d32a3251b4f69dae8646dfbedbc,6d394722d5fc5e721aee6875a218d8db
26443,436.24,2d54c241a27d5c40c9adf389868e0995,3895f60f6e6a89e5cfb7b72ffdcdf7e0
...,...,...,...
29232,0.00,03a81070a9a6e3bc2c4209e564764204,0e50810aa0fdb3784e55bda1bb0a5874
26896,0.00,f03c9c80de7a0a901cd92d8507717007,72b74e3ab8b2f7da203dbb50344879bb
40140,0.00,043b3cac6240fc110a721919bd6dea93,d87cf110269f58faa16ef95b748b702c
9300,0.00,6a7ca7365d53fad7a6b7dd45406112de,c9b9e60e5338143a868611fd96e30b8f


In [105]:
work=work.merge(fdp_max[['fdp_max','customer_unique_id']], how='left')
work

,customer_unique_id,geolocation_lng,geolocation_lat,nb_orders,mean_review_score,most_freq_payment_type,nb_voucher,voucher_value,mean_payment_installments,nb_days_last_order,most_freq_cat_name,payment_value,average_basket,min_basket,max_basket,sum_fdp,mean_fdp,fdp_min,fdp_max
0,861eff4711a542e4b93843c6dd7febb0,-47.398437,-20.534057,1.0,4.0,credit_card,0.0,0.0,2.0,259.0,office_furniture,146.87,146.87,146.87,146.87,21.88,21.88,21.88,21.88
1,290c77bc529b7ac935b93aa66c333dc3,-46.563869,-23.707626,1.0,5.0,credit_card,0.0,0.0,8.0,18.0,housewares,335.48,335.48,335.48,335.48,46.48,46.48,46.48,46.48
2,060e732b5b29e8181a18229c7b0b2b5e,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,259dac757896d24d7702b9acbbff3f3c,-46.193888,-23.530402,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,345ecd01c38d18a9036ed96c73b8d066,-47.068536,-22.900861,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96091,1a29b476fee25c95fbafc67c5ac95cf8,-46.633242,-23.570860,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96092,d52a67c98be1cf6a5c84435bd38d095d,-46.780156,-23.620738,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96093,e9f50caf99f032f0bf3c55141f019d99,-38.532430,-3.763108,0.0,-1.0,missing,0.0,0.0,0.0,NaN,missing,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96094,73c2643a0a458b49f58cea58833b192e,-51.174495,-29.921656,1.0,5.0,credit_card,0.0,0.0,2.0,88.0,watches_gifts,711.07,711.07,711.07,711.07,22.07,22.07,22.07,22.07


## C - Enregistrement fichier de travail <a name=C ></a>

In [106]:
filepath = 'Data/Clean/work_' + nom_fichier 
filepath

'Data/Clean/work_2017-02-01_to_2018-01-31.csv'

In [107]:
work.to_csv(filepath, index = False)